In [1]:
#import the necessary modules 
%matplotlib inline 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import scipy
import sklearn
import itertools 
from itertools import cycle 
import os.path as op
import timeit 
import json
import math

In [2]:
import multiprocessing as m_proc
m_proc.cpu_count()

4

In [3]:
# Import MDAnalysis
import MDAnalysis as mda
import statsmodels as stats
from MDAnalysis.analysis import distances, rdf

In [ ]:
from plgapeg_MD_analysis import (pers_length, get_rg_pers_poly, bavg_pers_cnt, pos_bead_autocorr_RA,
                                 rouse_relax, zimm_relax_fit, zimm_relax_func, rh_block_avg, mean_sq_e2e, hydro_rad_poly,
                                obs_autocorr_RA)

In [ ]:
from polymer_MD_analysis import (pers_length, get_rg_pers_poly, bavg_pers_cnt, pos_bead_autocorr_RA,
                                 rouse_relax, zimm_relax_fit, zimm_relax_func, rh_block_avg, mean_sq_e2e, hydro_rad_poly,
                                obs_autocorr_RA, getPairStats, pair_regression, simple_regression_conf)

## PLGA-PEG/water system analysis

### N = 6 PLGA-PEG/water 

In [4]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n6_plgapeg_wat = mda.Universe("n6plga_peg_wat/n6plgapeg_WAT.pdb", "n6plga_peg_wat/NoWat_n6plgapeg.xtc")

In [5]:
n6_plgapeg_wat.trajectory

<XTCReader n6plga_peg_wat/NoWat_n6plgapeg.xtc with 36001 frames of 69 atoms>

In [6]:
len(n6_plgapeg_wat.trajectory)

36001

In [7]:
#Select the polymer heavy atoms 
plgapeg_n6wat = n6_plgapeg_wat.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [8]:
np.unique(plgapeg_n6wat.resids)

array([1, 2, 3, 4, 5, 6])

In [ ]:
# Get cosine theta values at frame 0 
crv_n6plgapeg_wat = pers_length(plgapeg_n6wat,6)
crv_n6plgapeg_wat

### Orientation Order parameter vs. time 

In [ ]:
n_mon = 6 
start = 0
end = 36001

In [ ]:
OOp_n6plgapeg_wat = orientation_order_param(plgapeg_n6wat, n6_plgapeg_wat, n_mon, start, end)

In [ ]:
OOp_n6plgapeg_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_wat,linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 6 PLGA-PEG in water

In [ ]:
OOp_n6plgapeg_wat[82]

In [ ]:
OOp_n6plgapeg_wat[239]

In [ ]:
np.mean(OOp_n6plgapeg_wat)

In [ ]:
np.std(OOp_n6plgapeg_wat)

### Mean squared end to end distance 

In [ ]:
n_mon = 6 
start = 0
end = 36001

In [ ]:
eVec_n6plgapeg_wat, e2edis_n6plgapeg_wat = mean_sq_e2e(plgapeg_n6wat, n6_plgapeg_wat, n_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_wat[1]/10,linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,5)

#### Ree Autocorrelation function

In [ ]:
e2edis_n6plgapeg_wat[1]

In [ ]:
t_corr = 2000
window_shift = 20

In [ ]:
ACn6plgapegwat_Ree, eqvar_n6plgapegwat = obs_autocorr_RA(e2edis_n6plgapeg_wat[1],t_corr, window_shift,start,end)

In [ ]:
ACn6plgapegwat_Ree

In [ ]:
eqvar_n6plgapegwat

In [ ]:
ACn6plgapegwat_sqRee, evsqRee_n6plgapegwat = obs_autocorr_RA(e2edis_n6plgapeg_wat[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n6plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Ree[1]/100, ACn6plgapegwat_Ree[0],linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
n_mon = 6 
start = 0
end = 36001

In [ ]:
plgapegn6wat_rh = hydro_rad_poly(plgapeg_n6wat, n6_plgapeg_wat, n_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6wat_rh/10,linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,2)

#### Rh autocorrelation

In [ ]:
ACn6plgapegwatRh, eqvarRh_n6plgapegwat = obs_autocorr_RA(plgapegn6wat_rh,t_corr, window_shift,start,end)

In [ ]:
ACn6plgapegwatRh

In [ ]:
eqvarRh_n6plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegwatRh[1]/100, ACn6plgapegwatRh[0],linewidth=2, color='teal')
#plt.plot(A2n6plgapegwat_Ree[1]/100, ACn6plgapegwat_Ree[0],linewidth=2, color='r')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,20)
#plt.ylim(-0.2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n_mon = 6 
start = 0
end = 36001
t_corr = 1000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn6wat, tcSUM_plgapegn6wat = pos_bead_autocorr_RA(plgapeg_n6wat, n6_plgapeg_wat, n_mon, t_corr, window_shift, start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn6wat

In [ ]:
tcSUM_plgapegn6wat.shape

### Fitting autocorrelation data 

In [ ]:
tcRA_plgapegn6wat = np.load('n6ppwat_CtRA.npy', allow_pickle=True)
tcRA_plgapegn8wat = np.load('n8ppwat_CtRA.npy', allow_pickle=True)
tcRA_plgapegn10wat = np.load('n10ppwat_CtRA.npy', allow_pickle=True)
tcRA_plgapegn16wat = np.load('n16ppwat_CtRA.npy', allow_pickle=True)
tcRA_plgapegn20wat = np.load('n20ppwat_CtRA.npy', allow_pickle=True)
tcRA_plgapegn30wat = np.load('n30ppwat_CtRA.npy', allow_pickle=True)

In [ ]:
s_n6 =[2 for i in range(tcRA_plgapegn6wat[1].shape[0])]
s_n8 =[2 for i in range(tcRA_plgapegn8wat[1].shape[0])]
s_n10 =[2 for i in range(tcRA_plgapegn10wat[1].shape[0])]
s_n16 =[2 for i in range(tcRA_plgapegn16wat[1].shape[0])]
s_n20 =[2 for i in range(tcRA_plgapegn20wat[1].shape[0])]
s_n30 =[2 for i in range(tcRA_plgapegn30wat[1].shape[0])]

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(tcRA_plgapegn6wat[1]/100, tcRA_plgapegn6wat[0], color='teal', s=s_n6, label=r'$N_{PLGA-PEG}$ = 6')
plt.scatter(tcRA_plgapegn8wat[1]/100, tcRA_plgapegn8wat[0], color='dodgerblue', s=s_n8, label=r'$N_{PLGA-PEG}$ = 8')
plt.scatter(tcRA_plgapegn10wat[1]/100, tcRA_plgapegn10wat[0], color='darkmagenta', s=s_n10, label=r'$N_{PLGA-PEG}$ = 10')
plt.scatter(tcRA_plgapegn16wat[1]/100, tcRA_plgapegn16wat[0], color='maroon', s=s_n16, label=r'$N_{PLGA-PEG}$ = 16')
plt.scatter(tcRA_plgapegn20wat[1]/100, tcRA_plgapegn20wat[0], color='orangered', s=s_n20, label=r'$N_{PLGA-PEG}$ = 20')
plt.scatter(tcRA_plgapegn30wat[1]/100, tcRA_plgapegn30wat[0], color='darkolivegreen', s=s_n30, label=r'$N_{PLGA-PEG}$ = 30')
plt.title(r'Positional bead autocorrelation PLGA-PEG in Water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
xdata_plgapegn6wat = tcRA_plgapegn6wat[1]/100
ydata_plgapegn6wat = tcRA_plgapegn6wat[0]

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.title(r'Positional bead autocorrelation PLGA-PEG in Water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,30)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rouse_relax(1, 2, 6)

In [ ]:
ts = 1
hs = 0.1
n_mon = 6
n_b = n_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
zimm_relax_func(1, 2, 0.1, 5)

In [ ]:
def res_plgapegn6wat(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 5
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
from lmfit import Minimizer, Parameters, report_fit

In [ ]:
ppfit_n6wat = Parameters()
#ppfit_n6wat.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n6wat.add(name='t_first', value=90)

In [ ]:
mini_ppn6wat = Minimizer(res_plgapegn6wat, ppfit_n6wat, fcn_args=(xdata_plgapegn6wat, ydata_plgapegn6wat))
out_ppn6wat = mini_ppn6wat.leastsq()
#bfit_n10 = ydata_n10wat + out_n10wat.residual 

In [ ]:
report_fit(out_ppn6wat.params)

In [ ]:
out_ppn6wat.params

In [ ]:
# with h varying, tr = 0.25790932 +/- 0.00187053, h* = 0.25372179 +/- 0.01834767
twat_n6plgapeg = []
n_bonds = 5
for i in range(len(xdata_plgapegn6wat)):
    twat_n6plgapeg.append(zimm_relax_fit(xdata_plgapegn6wat[i], 0.25745834, 0.26,  n_bonds))


In [ ]:
t_c, tz_n6plgapeg, zimm_n6plgapeg, rouse_n6plgapeg = zimm_relax_func(1, 0.25745834, 0.26,  n_bonds)

In [ ]:
x_pos = np.arange(5)
width = 0.25
aa_types = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$"]
fig = plt.figure(figsize=(7,7))
plt.bar(x_pos, tz_n6plgapeg, width,ecolor='black',capsize=5, color='c')
plt.xticks(x_pos, labels=aa_types, fontsize=12)
plt.ylabel(r'Relaxation times', fontsize=15)

In [ ]:
x_pos = np.arange(5)
width = 0.25
aa_eigen_ppn6 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA PEG in water N = 6', fontsize=18)
plt.bar(x_pos, rouse_n6plgapeg, width, ecolor='black',capsize=5, color='c', label='Rouse' )
plt.bar(x_pos+width, zimm_n6plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos, labels=aa_eigen_ppn6, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.plot(xdata_plgapegn6wat, twat_n6plgapeg, color='teal')
plt.title(r'Positional bead autocorrelation PLGA PEG in water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,30)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppwat_msen6 = np.array([ydata_plgapegn6wat - np.array(twat_n6plgapeg)])
#rpwat_msen6

In [ ]:
plt.figure(figsize=(7,7))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, rppwat_msen6, color='teal', s=s_n6)
plt.title(r'Relaxation time Fitting Residuals PLGA-PEG in water', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,15])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

### Radius of Gyration vs. time N = 6 PLGA-PEG in water

In [ ]:
# Key variables 
n_mon = 6 
start = 0
end = 36001

In [ ]:
n6plgapeg_rgens_wat, cor_n6plgapeg_wat, N6plgapeg_cos_wat, rgwat_n6plgapeg = get_rg_pers_poly(plgapeg_n6wat, n6_plgapeg_wat,
                                                                                                  start,end)

In [ ]:
n6plgapeg_rgens_wat[0].shape

In [ ]:
cor_n6plgapeg_wat[3]

In [ ]:
N6plgapeg_cos_wat

In [ ]:
rgwat_n6plgapeg

In [ ]:
n6plgapeg_rgens_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_wat[0]/10,linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,2)

#### Rg Autocorrelation

In [ ]:
ACn6plgapegwat_Rg, eqvarRg_n6plgapegwat = obs_autocorr_RA(n6plgapeg_rgens_wat[0],t_corr, window_shift,start,end)

In [ ]:
ACn6plgapegwat_Rg

In [ ]:
eqvarRg_n6plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG in water autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Rg[1]/100, ACn6plgapegwat_Rg[0],linewidth=2, color='teal')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

#### Get average COM distance, Arc length values

In [9]:
cb_n6plga = np.zeros(shape=(36001))
cb_n6pplinker = np.zeros(shape=(36001))
cb_n6peg = np.zeros(shape=(36001))
count = 0
for ts in n6_plgapeg_wat.trajectory[0:36001]:
    # 1st two PLGA residues
    n6m1_plga = n6_plgapeg_wat.select_atoms("resid 1")
    n6m2_plga = n6_plgapeg_wat.select_atoms("resid 2")
    # PLGA-PEG linker
    n6pl_link = n6_plgapeg_wat.select_atoms("resid 3")
    n6peg_link = n6_plgapeg_wat.select_atoms("resid 4")
    # last two PEG residues 
    n6l1_peg = n6_plgapeg_wat.select_atoms("resid 5")
    n6l2_peg = n6_plgapeg_wat.select_atoms("resid 6")
    
    cb_n6plga[count] = mda.analysis.distances.distance_array(n6m1_plga.center_of_mass(), n6m2_plga.center_of_mass(), 
                                                   box=n6_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n6pplinker[count] = mda.analysis.distances.distance_array(n6pl_link.center_of_mass(), n6peg_link.center_of_mass(), 
                                                   box=n6_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n6peg[count] = mda.analysis.distances.distance_array(n6l1_peg.center_of_mass(), n6l2_peg.center_of_mass(), 
                                                   box=n6_plgapeg_wat.trajectory.ts.dimensions)
    
    count += 1
    

    

In [10]:
COM_n6pp = np.zeros(shape=(3,2))
COM_n6pp[0,0] = np.mean(cb_n6plga)
COM_n6pp[1,0] = np.mean(cb_n6pplinker)
COM_n6pp[2,0] = np.mean(cb_n6peg)
COM_n6pp[0,1] = np.std(cb_n6plga)
COM_n6pp[1,1] = np.std(cb_n6pplinker)
COM_n6pp[2,1] = np.std(cb_n6peg)
COM_n6pp

array([[5.39885887, 0.72308498],
       [4.19863753, 0.37848645],
       [3.24377771, 0.15432754]])

#### Correlation values at each arc length from the whole 360 ns trajectory, N = 6 PLGA-PEG/water

In [ ]:
# x values
blen_n6pp = np.zeros(shape=5)
for i in range(n_mon-1):
    if i < int((n_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_n6pp[i] = 0
        elif i != 0:
            blen_n6pp[i] = np.sum(blen_n6pp[i-1]) + COM_n6pp[0,0]    
    elif i == int((n_mon/2)-1):
        #PLGA-PEG linker length
        blen_n6pp[i] = np.sum(blen_n6pp[i-1]) + COM_n6pp[1,0]
    elif i > int((n_mon/2)-1):
    #    # PEG contour length values
        blen_n6pp[i] = np.sum(blen_n6pp[i-1]) + COM_n6pp[2,0]
        
blen_n6pp

In [ ]:
# Error prop. into natural log std deviation
mk_n6ppwat = cor_n6plgapeg_wat[1]/cor_n6plgapeg_wat[0]
mk_n6ppwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n6pp, np.log(cor_n6plgapeg_wat[0]), yerr=mk_n6ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in water
n6_blksplgapeg_wat , n6plgapeg_lpwat = bavg_pers_cnt(5, plgapeg_n6wat, n6_plgapeg_wat, blen_n6pp, 3, 0 , 36001)

In [ ]:
n6_blksplgapeg_wat

In [ ]:
n6plgapeg_lpwat

In [ ]:
n6plgapeg_lpwat[2]

In [ ]:
np.mean(n6plgapeg_lpwat[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
def line_fit(slope, x):
    return slope*x 

In [ ]:
blen_n6pp

In [ ]:
gg_n6plgapeg_wat = line_fit(np.mean(n6plgapeg_lpwat[2]),blen_n6pp)

In [ ]:
gg_n6plgapeg_wat

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 6 PLGA-PEG/water

In [12]:
blppn6 = []
n_mon = 6
for i in range(n_mon-1):
    if i < int((n_mon/2)-1):
        # PLGA contour length values
        blppn6.append(COM_n6pp[0,0])    
    elif i == int((n_mon/2)-1):
        #PLGA-PEG linker length
        blppn6.append(COM_n6pp[1,0]) 
    elif i > int((n_mon/2)-1):
    #    # PEG contour length values
        blppn6.append(COM_n6pp[2,0]) 

# Contour Length, N = 6 PLGA-PEG in water
lc_ppn6 = np.sum(blppn6)
lc_ppn6

21.483910690614188

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegwat = np.zeros(shape=(3,6))
re2e_rgsq_plgapegwat[1,0] = lc_ppn6
re2e_rgsq_plgapegwat

In [ ]:
a_ppn6 = np.mean(n6_blksplgapeg_wat['Avg Sq. end to end distance'])

In [ ]:
b_ppn6 = np.mean(n6_blksplgapeg_wat['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegwat[0,0] = a_ppn6/b_ppn6
re2e_rgsq_plgapegwat

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegwat = np.zeros(shape=(3,6))
rhy_rg_plgapegwat[1,0] = lc_ppn6
rhy_rg_plgapegwat

In [ ]:
aa_ppn6 = np.mean(n6_blksplgapeg_wat['Avg Radius of gyration'])

In [ ]:
bb_ppn6 = np.mean(n6_blksplgapeg_wat['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegwat[0,0] = aa_ppn6/bb_ppn6
rhy_rg_plgapegwat

In [ ]:
# Kuhn length array at different monomer lengths
kuhnlpL_plgapegwat = np.zeros(shape=(3,6))
kuhnlpL_plgapegwat[1] = np.array([6,8,10,16,20,30])
kuhnlpL_plgapegwat

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegwat[0,0] = np.mean(n6_blksplgapeg_wat["Avg persistence length"])/lc_ppn6
kuhnlpL_plgapegwat

#### Block averaged Radius of gyration and persistence length, N = 6 PLGA-PEG/water

In [ ]:
np.mean(n6_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.std(n6_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.mean(n6_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
np.std(n6_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n6pp, np.log(cor_n6plgapeg_wat[0]), yerr=mk_n6ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_n6pp, gg_n6plgapeg_wat, color='teal')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) in water', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
plt.legend([r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 19.4 $\AA$ ± 1.43 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
#plt.text(0.5, -6.94,r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.61 $\AA$', fontsize=15, color='#1F2E69')

In [ ]:
rgplgapeg_olig_wat =  pd.DataFrame(data=n6_blksplgapeg_wat["Avg Radius of gyration"], columns=['$R_{g}$ [Angstrom] N = 6 PLGA-PEG water'])
rgplgapeg_olig_wat

In [ ]:
pers_plgapegt_wat = pd.DataFrame(data=n6_blksplgapeg_wat["Avg persistence length"], columns=[r"$L_{p}$ [Angstrom] N = 6 PLGA-PEG water"])
pers_plgapegt_wat

### N = 8 PLGA-PEG/water 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n8_plgapeg_wat = mda.Universe("n8plga_peg_wat/n8plgapeg_WAT.pdb", "n8plga_peg_wat/NoWat_n8plgapeg.xtc")

In [ ]:
n8_plgapeg_wat.trajectory

In [ ]:
len(n8_plgapeg_wat.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n8wat = n8_plgapeg_wat.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n8wat.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n8plgapeg_wat = pers_length(plgapeg_n8wat,8)
crv_n8plgapeg_wat

### Orientation Order parameter vs. time 

In [ ]:
n8_mon = 8 
start = 0
end = 36001

In [ ]:
OOp_n8plgapeg_wat = orientation_order_param(plgapeg_n8wat, n8_plgapeg_wat, n8_mon, start, end)

In [ ]:
OOp_n8plgapeg_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_wat,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_wat,linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 8 PLGA-PEG in water

In [ ]:
OOp_n8plgapeg_wat[82]

In [ ]:
OOp_n8plgapeg_wat[239]

In [ ]:
np.mean(OOp_n8plgapeg_wat)

In [ ]:
np.std(OOp_n8plgapeg_wat)

### Mean squared end to end distance 

In [ ]:
eVec_n8plgapeg_wat, e2edis_n8plgapeg_wat = mean_sq_e2e(plgapeg_n8wat, n8_plgapeg_wat, n8_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_wat[1]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, e2edis_n8plgapeg_wat[1]/10,linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,7)

#### Ree Autocorrelation function

In [ ]:
e2edis_n8plgapeg_wat[1]

In [ ]:
ACn8plgapegwat_Ree, eqvar_n8plgapegwat = obs_autocorr_RA(e2edis_n8plgapeg_wat[1],t_corr, window_shift,start,end)

In [ ]:
ACn8plgapegwat_Ree

In [ ]:
eqvar_n8plgapegwat

In [ ]:
ACn8plgapegwat_sqRee, evsqRee_n8plgapegwat = obs_autocorr_RA(e2edis_n8plgapeg_wat[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n8plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Ree[1]/100, ACn6plgapegwat_Ree[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegwat_Ree[1]/100, ACn8plgapegwat_Ree[0],linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn8wat_rh = hydro_rad_poly(plgapeg_n8wat, n8_plgapeg_wat, n8_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6wat_rh/10,linewidth=2, color='teal')
plt.plot(trj_len/100, plgapegn8wat_rh/10,linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,7)

#### Rh autocorrelation

In [ ]:
t_corr = 4000
window_shift = 15

In [ ]:
ACn8plgapegwatRh, eqvarRh_n8plgapegwat = obs_autocorr_RA(plgapegn8wat_rh,t_corr, window_shift,start,end)

In [ ]:
ACn8plgapegwatRh

In [ ]:
eqvarRh_n8plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegwatRh[1]/100, ACn6plgapegwatRh[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegwatRh[1]/100, ACn8plgapegwatRh[0],linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in water','N = 8 in water'], frameon=False, fontsize=14)
plt.xlim(0,20)
#plt.ylim(-0.2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n8_mon = 8 
start = 0
end = 36001
t_corr = 1000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn8wat, tcSUM_plgapegn8wat = pos_bead_autocorr_RA(plgapeg_n8wat, n8_plgapeg_wat, n8_mon, t_corr, window_shift, start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn8wat

In [ ]:
tcSUM_plgapegn8wat.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn8wat = tcRA_plgapegn8wat[1]/100
ydata_plgapegn8wat = tcRA_plgapegn8wat[0]

In [ ]:
ydata_plgapegn8wat.shape

In [ ]:
xdata_plgapegn8wat.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.title(r'Positional bead autocorrelation PLGA-PEG in Water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,30)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rouse_relax(1, 2, 6)

In [ ]:
ts = 1
hs = 0.1
n8_mon = 8
n_b = n8_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
zimm_relax_func(ts,tr1,hs,n_b)

In [ ]:
def res_plgapegn8wat(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 7
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n8wat = Parameters()
#ppfit_n8wat.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n8wat.add(name='t_first', value=2)

In [ ]:
mini_ppn8wat = Minimizer(res_plgapegn8wat, ppfit_n8wat, fcn_args=(xdata_plgapegn8wat, ydata_plgapegn8wat))
out_ppn8wat = mini_ppn8wat.leastsq()
#bfit_n10 = ydata_n10wat + out_n10wat.residual 

In [ ]:
report_fit(out_ppn8wat.params)

In [ ]:
out_ppn8wat.params

In [ ]:
# with h varying, tr = 0.39469879 +/- 0.00246941,h* = 0.25999921 +/- 0.04222828
twat_n8plgapeg = []
n_bonds = 7
for i in range(len(xdata_plgapegn8wat)):
    twat_n8plgapeg.append(zimm_relax_fit(xdata_plgapegn8wat[i], 0.39577315, 0.26,  n_bonds))


In [ ]:
t_c, tz_n8plgapeg, zimm_n8plgapeg, rouse_n8plgapeg = zimm_relax_func(1, 0.39577315, 0.26,  n_bonds)

In [ ]:
x_pos_n6 = np.arange(5)
x_pos_n8 = np.arange(7)
width = 0.25
aa_types_n8 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in water', fontsize=18, y=1.03)
plt.bar(x_pos_n6+width, tz_n6plgapeg, width, ecolor='black',capsize=5, color='teal', label="N = 6")
plt.bar(x_pos_n8, tz_n8plgapeg, width, ecolor='black',capsize=5, color='dodgerblue', label='N = 8')
plt.xticks(x_pos_n6+width, labels=aa_types, fontsize=12)
plt.xticks(x_pos_n8, labels=aa_types_n8, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
aa_eigen_n8 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in water N = 8', fontsize=18)
plt.bar(x_pos_n8, rouse_n8plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n8+width, zimm_n8plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n8+width,labels=aa_eigen_n8, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.plot(xdata_plgapegn6wat, twat_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8wat, twat_n8plgapeg, color='dodgerblue')
plt.title(r'Positional bead autocorrelation PLGA-PEG in water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,30)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppwat_msen8 = np.array([ydata_plgapegn8wat - np.array(twat_n8plgapeg)])
#rpwat_msen6

In [ ]:
plt.figure(figsize=(7,7))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, rppwat_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, rppwat_msen8, color='dodgerblue', s=s_n8)
plt.title(r'Relaxation time Fitting Residuals PlGA-PEG in water', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,10])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Radius of Gyration vs. time N = 8 PLGA-PEG in water

In [ ]:
# Key variables 
n8_mon = 8 
start = 0
end = 36001

In [ ]:
n8plgapeg_rgens_wat, cor_n8plgapeg_wat, N8plgapeg_cos_wat, rgwat_n8plgapeg = get_rg_pers_poly(plgapeg_n8wat, n8_plgapeg_wat,
                                                                                                  start,end)

In [ ]:
n8plgapeg_rgens_wat[0].shape

In [ ]:
cor_n8plgapeg_wat[3]

In [ ]:
N8plgapeg_cos_wat

In [ ]:
rgwat_n8plgapeg

In [ ]:
n8plgapeg_rgens_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_wat[0]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, n8plgapeg_rgens_wat[0]/10,linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,2)

#### Rg Autocorrelation

In [ ]:
t_corr = 4000
window_shift = 15

In [ ]:
ACn8plgapegwat_Rg, eqvarRg_n8plgapegwat = obs_autocorr_RA(n8plgapeg_rgens_wat[0],t_corr, window_shift,start,end)

In [ ]:
ACn8plgapegwat_Rg

In [ ]:
eqvarRg_n8plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in water autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Rg[1]/100, ACn6plgapegwat_Rg[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegwat_Rg[1]/100, ACn8plgapegwat_Rg[0],linewidth=2, color='dodgerblue')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

#### Get average COM distance, Contour length calc

In [ ]:
cb_n8plga = np.zeros(shape=(36001))
cb_n8pplinker = np.zeros(shape=(36001))
cb_n8peg = np.zeros(shape=(36001))
count = 0
for ts in n8_plgapeg_wat.trajectory[0:36001]:
    # 1st two PLGA residues
    n8m1_plga = n8_plgapeg_wat.select_atoms("resid 1")
    n8m2_plga = n8_plgapeg_wat.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n8pl_link = n8_plgapeg_wat.select_atoms("resid 4")
    n8peg_link = n8_plgapeg_wat.select_atoms("resid 5")
    # last two PEG residues 
    n8l1_peg = n8_plgapeg_wat.select_atoms("resid 7")
    n8l2_peg = n8_plgapeg_wat.select_atoms("resid 8")
    
    cb_n8plga[count] = mda.analysis.distances.distance_array(n8m1_plga.center_of_mass(), n8m2_plga.center_of_mass(), 
                                                   box=n8_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n8pplinker[count] = mda.analysis.distances.distance_array(n8pl_link.center_of_mass(), n8peg_link.center_of_mass(), 
                                                   box=n8_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n8peg[count] = mda.analysis.distances.distance_array(n8l1_peg.center_of_mass(), n8l2_peg.center_of_mass(), 
                                                   box=n8_plgapeg_wat.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_n8pp = np.zeros(shape=(3,2))
COM_n8pp[0,0] = np.mean(cb_n8plga)
COM_n8pp[1,0] = np.mean(cb_n8pplinker)
COM_n8pp[2,0] = np.mean(cb_n8peg)
COM_n8pp[0,1] = np.std(cb_n8plga)
COM_n8pp[1,1] = np.std(cb_n8pplinker)
COM_n8pp[2,1] = np.std(cb_n8peg)
COM_n8pp

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 8 PLGA-PEG/water

In [ ]:
# x values
blen_n8pp = np.zeros(shape=7)
for i in range(n8_mon-1):
    if i < int((n8_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_n8pp[i] = 0
        elif i != 0:
            blen_n8pp[i] = np.sum(blen_n8pp[i-1]) + COM_n8pp[0,0]    
    elif i == int((n8_mon/2)-1):
        #PLGA-PEG linker length
        blen_n8pp[i] = np.sum(blen_n8pp[i-1]) + COM_n8pp[1,0]
    elif i > int((n8_mon/2)-1):
    #    # PEG contour length values
        blen_n8pp[i] = np.sum(blen_n8pp[i-1]) + COM_n8pp[2,0]
        
blen_n8pp

In [ ]:
# Error prop. into natural log std deviation
mk_n8ppwat = cor_n8plgapeg_wat[1]/cor_n8plgapeg_wat[0]
mk_n8ppwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n8pp, np.log(cor_n8plgapeg_wat[0]), yerr=mk_n8ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in water
n8_blksplgapeg_wat , n8plgapeg_lpwat = bavg_pers_cnt(5, plgapeg_n8wat, n8_plgapeg_wat, blen_n8pp, 4, 0 , 36001)

In [ ]:
n8_blksplgapeg_wat

In [ ]:
n8plgapeg_lpwat

In [ ]:
n8plgapeg_lpwat[2]

In [ ]:
np.mean(n8plgapeg_lpwat[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_n8pp

In [ ]:
gg_n8plgapeg_wat = line_fit(np.mean(n8plgapeg_lpwat[2]),blen_n8pp)

In [ ]:
gg_n6plgapeg_n8wat = line_fit(np.mean(n6plgapeg_lpwat[2]),blen_n8pp)

In [ ]:
gg_n8plgapeg_wat

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 8 PLGA-PEG in water

In [ ]:
blppn8 = []
for i in range(n8_mon-1):
    if i < int((n8_mon/2)-1):
        # PLGA contour length values
        blppn8.append(COM_n8pp[0,0])    
    elif i == int((n8_mon/2)-1):
        #PLGA-PEG linker length
        blppn8.append(COM_n8pp[1,0]) 
    elif i > int((n8_mon/2)-1):
    #    # PEG contour length values
        blppn8.append(COM_n8pp[2,0]) 

# Contour Length, N = 6 PLGA-PEG in water
lc_ppn8 = np.sum(blppn8)
lc_ppn8

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegwat[1,1] = lc_ppn8
re2e_rgsq_plgapegwat

In [ ]:
a_ppn8 = np.mean(n8_blksplgapeg_wat['Avg Sq. end to end distance'])

In [ ]:
b_ppn8 = np.mean(n8_blksplgapeg_wat['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegwat[0,1] = a_ppn8/b_ppn8
re2e_rgsq_plgapegwat

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegwat[1,1] = lc_ppn8
rhy_rg_plgapegwat

In [ ]:
aa_ppn8 = np.mean(n8_blksplgapeg_wat['Avg Radius of gyration'])

In [ ]:
bb_ppn8 = np.mean(n8_blksplgapeg_wat['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegwat[0,1] = aa_ppn8/bb_ppn8
rhy_rg_plgapegwat

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegwat[0,1] = np.mean(n8_blksplgapeg_wat["Avg persistence length"])/lc_ppn8
kuhnlpL_plgapegwat

#### Block averaged Radius of gyration and persistence length, N = 8 PLGA-PEG in water

In [ ]:
np.mean(n8_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.std(n8_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.mean(n8_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
np.std(n8_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n6pp, np.log(cor_n6plgapeg_wat[0]), yerr=mk_n6ppwat, color='teal', linestyle="None",marker='o',
             capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n8pp, np.log(cor_n8plgapeg_wat[0]), yerr=mk_n8ppwat, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_n8pp, gg_n6plgapeg_n8wat, color='teal')
plt.plot(blen_n8pp, gg_n8plgapeg_wat, color='dodgerblue')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in water', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-6,1)
plt.xlim(0,30)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -5.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 19.4 $\AA$ ± 1.43 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -5.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 20.8 $\AA$ ± 1.03 $\AA$', fontsize=15, color='dodgerblue')

In [ ]:
rgplgapeg_olig_wat[r"$R_{g}$ [Angstrom] N = 8 PLGA-PEG water"] = n8_blksplgapeg_wat["Avg Radius of gyration"]
rgplgapeg_olig_wat

In [ ]:
pers_plgapegt_wat[r"$L_{p}$ [Angstrom] N = 8 PLGA-PEG water"] = n8_blksplgapeg_wat["Avg persistence length"]
pers_plgapegt_wat

### N = 10 PLGA-PEG/water 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n10_plgapeg_wat = mda.Universe("n10plga_peg_wat/n10plgapeg_WAT.pdb", "n10plga_peg_wat/NoWat_n10plgapeg.xtc")

In [ ]:
n10_plgapeg_wat.trajectory

In [ ]:
len(n10_plgapeg_wat.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n10wat = n10_plgapeg_wat.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n10wat.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n10plgapeg_wat = pers_length(plgapeg_n10wat,10)
crv_n10plgapeg_wat

### Orientation Order parameter vs. time 

In [ ]:
n10_mon = 10 
start = 0
end = 36001

In [ ]:
OOp_n10plgapeg_wat = orientation_order_param(plgapeg_n10wat, n10_plgapeg_wat, n10_mon, start, end)

In [ ]:
OOp_n10plgapeg_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_wat,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_wat,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, OOp_n10plgapeg_wat,linewidth=2, color='darkmagenta')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 10 PLGA-PEG in water

In [ ]:
OOp_n10plgapeg_wat[82]

In [ ]:
OOp_n10plgapeg_wat[239]

In [ ]:
np.mean(OOp_n10plgapeg_wat)

In [ ]:
np.std(OOp_n10plgapeg_wat)

### Mean squared end to end distance 

In [ ]:
eVec_n10plgapeg_wat, e2edis_n10plgapeg_wat = mean_sq_e2e(plgapeg_n10wat, n10_plgapeg_wat, n10_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_wat[1]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, e2edis_n8plgapeg_wat[1]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, e2edis_n10plgapeg_wat[1]/10,linewidth=2, color='darkmagenta')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,5)

#### Ree Autocorrelation function

In [ ]:
e2edis_n10plgapeg_wat[1]

In [ ]:
t_corr = 7000
window_shift = 9

In [ ]:
ACn10plgapegwat_Ree, eqvar_n10plgapegwat = obs_autocorr_RA(e2edis_n10plgapeg_wat[1],t_corr, window_shift,start,end)

In [ ]:
ACn10plgapegwat_Ree

In [ ]:
eqvar_n10plgapegwat

In [ ]:
ACn10plgapegwat_sqRee, evsqRee_n10plgapegwat = obs_autocorr_RA(e2edis_n10plgapeg_wat[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n10plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Ree[1]/100, ACn6plgapegwat_Ree[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwat_Ree[1]/100, ACn8plgapegwat_Ree[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwat_Ree[1]/100, ACn10plgapegwat_Ree[0],linewidth=2, color='#8C52FC')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree','N = 10 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn10wat_rh = hydro_rad_poly(plgapeg_n10wat, n10_plgapeg_wat, n10_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6wat_rh/10,linewidth=2, color='teal')
plt.plot(trj_len/100, plgapegn8wat_rh/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, plgapegn10wat_rh/10,linewidth=2, color='darkmagenta')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,2)

#### Rh autocorrelation

In [ ]:
t_corr = 10000
window_shift = 9

In [ ]:
ACn10plgapegwatRh, eqvarRh_n10plgapegwat = obs_autocorr_RA(plgapegn10wat_rh,t_corr, window_shift,start,end)

In [ ]:
ACn10plgapegwatRh

In [ ]:
eqvarRh_n10plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegwatRh[1]/100, ACn6plgapegwatRh[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwatRh[1]/100, ACn8plgapegwatRh[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwatRh[1]/100, ACn10plgapegwatRh[0],linewidth=2, color='#8C52FC')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in water','N = 8 in water','N = 10 in water'], frameon=False, fontsize=14)
plt.xlim(0,20)
#plt.ylim(-0.2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n10_mon = 10 
start = 0
end = 36001
t_corr = 10000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn10wat, tcSUM_plgapegn10wat = pos_bead_autocorr_RA(plgapeg_n10wat, n10_plgapeg_wat, n10_mon, t_corr, window_shift, start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn10wat

In [ ]:
tcSUM_plgapegn10wat.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn10wat = tcRA_plgapegn10wat[1]/100
ydata_plgapegn10wat = tcRA_plgapegn10wat[0]

In [ ]:
ydata_plgapegn10wat.shape

In [ ]:
xdata_plgapegn10wat.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, ydata_plgapegn10wat, color='darkmagenta', s=s_n10)
plt.title(r'Positional bead autocorrelation PLGA-PEG in Water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,100)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
ts = 1
hs = 0.1
n10_mon = 10
n_b = n10_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
rouse_relax(1, 2, 6)

In [ ]:
zimm_relax_func(1, 2, 0.1, 5)

In [ ]:
def res_plgapegn10wat(variabls, xnp, ynp):
    hs_np = variabls['h_star']
    #hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 9
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n10wat = Parameters()
ppfit_n10wat.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n10wat.add(name='t_first', value=90)

In [ ]:
mini_ppn10wat = Minimizer(res_plgapegn10wat, ppfit_n10wat, fcn_args=(xdata_plgapegn10wat, ydata_plgapegn10wat))
out_ppn10wat = mini_ppn10wat.leastsq()
#bfit_n10 = ydata_n10wat + out_n10wat.residual 

In [ ]:
report_fit(out_ppn10wat.params)

In [ ]:
out_ppn10wat.params

In [ ]:
# with h varying, tr = 0.49248736 +/- 0.00336387,h* = 0.25999384 +/- 0.02282596
twat_n10plgapeg = []
n_bonds = 9
for i in range(len(xdata_plgapegn10wat)):
    twat_n10plgapeg.append(zimm_relax_fit(xdata_plgapegn10wat[i], 0.49248736, 0.26,  n_bonds))


In [ ]:
t_c, tz_n10plgapeg, zimm_n10plgapeg, rouse_n10plgapeg = zimm_relax_func(1, 0.49248736, 0.26,  n_bonds)

In [ ]:
len(tz_n10plgapeg)

In [ ]:
x_pos_n6 = np.arange(5)
x_pos_n8 = np.arange(7)
x_pos_n10 = np.arange(9)
width = 0.25
aa_types_n10 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$",
           r"$\tau_{8}$", r"$\tau_{9}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in water', fontsize=18)
plt.bar(x_pos_n6+(2*width), tz_n6plgapeg, width, ecolor='black',capsize=5, color='teal', label="N = 6")
plt.bar(x_pos_n8+(width), tz_n8plgapeg, width, ecolor='black',capsize=5, color='dodgerblue', label='N = 8')
plt.bar(x_pos_n10, tz_n10plgapeg, width, ecolor='black',capsize=5, color='darkmagenta', label='N = 10')
plt.xticks(x_pos_n10, labels=aa_types_n10, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
aa_eigen_n10 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$",r"$a_{8}$",
           r"$a_{9}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in water N = 10', fontsize=18)
plt.bar(x_pos_n10, rouse_n10plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n10+width, zimm_n10plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n10+width,labels=aa_eigen_n10, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, ydata_plgapegn10wat, color='darkmagenta', s=s_n10)
plt.plot(xdata_plgapegn6wat, twat_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8wat, twat_n8plgapeg, color='dodgerblue')
plt.plot(xdata_plgapegn10wat, twat_n10plgapeg, color='darkmagenta')
plt.title(r'Positional bead autocorrelation PLGA-PEG in water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,100)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppwat_msen10 = np.array([ydata_plgapegn10wat - np.array(twat_n10plgapeg)])
rppwat_msen10

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, rppwat_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, rppwat_msen8, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, rppwat_msen10, color='darkmagenta', s=s_n10)
plt.title(r'Relaxation time Fitting Residuals PLGA in water', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,30])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Radius of Gyration vs. time N = 10 PLGA-PEG in water

In [ ]:
n10plgapeg_rgens_wat, cor_n10plgapeg_wat, N10plgapeg_cos_wat, rgwat_n10plgapeg = get_rg_pers_poly(plgapeg_n10wat, n10_plgapeg_wat,
                                                                                                  start,end)

In [ ]:
n10plgapeg_rgens_wat[0].shape

In [ ]:
cor_n10plgapeg_wat[3]

In [ ]:
N10plgapeg_cos_wat

In [ ]:
rgwat_n10plgapeg

In [ ]:
n10plgapeg_rgens_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_wat[0]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, n8plgapeg_rgens_wat[0]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, n10plgapeg_rgens_wat[0]/10,linewidth=2, color='darkmagenta')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water', 'N = 10 in Water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,2)

#### Rg Autocorrelation

In [ ]:
t_corr = 10000
window_shift = 9

In [ ]:
ACn10plgapegwat_Rg, eqvarRg_n10plgapegwat = obs_autocorr_RA(n10plgapeg_rgens_wat[0],t_corr, window_shift,start,end)

In [ ]:
ACn10plgapegwat_Rg

In [ ]:
eqvarRg_n10plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG in water autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Rg[1]/100, ACn6plgapegwat_Rg[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwat_Rg[1]/100, ACn8plgapegwat_Rg[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwat_Rg[1]/100, ACn10plgapegwat_Rg[0],linewidth=2, color='#8C52FC')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

#### Get average COM distance, arch length values

In [ ]:
cb_n10plga = np.zeros(shape=(36001))
cb_n10pplinker = np.zeros(shape=(36001))
cb_n10peg = np.zeros(shape=(36001))
count = 0
for ts in n10_plgapeg_wat.trajectory[0:36001]:
    # 1st two PLGA residues
    n10m1_plga = n10_plgapeg_wat.select_atoms("resid 1")
    n10m2_plga = n10_plgapeg_wat.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n10pl_link = n10_plgapeg_wat.select_atoms("resid 5")
    n10peg_link = n10_plgapeg_wat.select_atoms("resid 6")
    # last two PEG residues 
    n10l1_peg = n10_plgapeg_wat.select_atoms("resid 9")
    n10l2_peg = n10_plgapeg_wat.select_atoms("resid 10")
    
    cb_n10plga[count] = mda.analysis.distances.distance_array(n10m1_plga.center_of_mass(), n10m2_plga.center_of_mass(), 
                                                   box=n10_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n10pplinker[count] = mda.analysis.distances.distance_array(n10pl_link.center_of_mass(), n10peg_link.center_of_mass(), 
                                                   box=n10_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n10peg[count] = mda.analysis.distances.distance_array(n10l1_peg.center_of_mass(), n10l2_peg.center_of_mass(), 
                                                   box=n10_plgapeg_wat.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_n10pp = np.zeros(shape=(3,2))
COM_n10pp[0,0] = np.mean(cb_n10plga)
COM_n10pp[1,0] = np.mean(cb_n10pplinker)
COM_n10pp[2,0] = np.mean(cb_n10peg)
COM_n10pp[0,1] = np.std(cb_n10plga)
COM_n10pp[1,1] = np.std(cb_n10pplinker)
COM_n10pp[2,1] = np.std(cb_n10peg)
COM_n10pp

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 10 PLGA-PEG/water

In [ ]:
# x values
blen_n10pp = np.zeros(shape=9)
for i in range(n10_mon-1):
    if i < int((n10_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_n10pp[i] = 0
        elif i != 0:
            blen_n10pp[i] = np.sum(blen_n10pp[i-1]) + COM_n10pp[0,0]    
    elif i == int((n10_mon/2)-1):
        #PLGA-PEG linker length
        blen_n10pp[i] = np.sum(blen_n10pp[i-1]) + COM_n10pp[1,0]
    elif i > int((n10_mon/2)-1):
    #    # PEG contour length values
        blen_n10pp[i] = np.sum(blen_n10pp[i-1]) + COM_n10pp[2,0]
        
blen_n10pp

In [ ]:
# Error prop. into natural log std deviation
mk_n10ppwat = cor_n10plgapeg_wat[1]/cor_n10plgapeg_wat[0]
mk_n10ppwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n10pp, np.log(cor_n10plgapeg_wat[0]), yerr=mk_n10ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in water
n10_blksplgapeg_wat , n10plgapeg_lpwat = bavg_pers_cnt(5, plgapeg_n10wat, n10_plgapeg_wat, blen_n10pp, 5, 0 , 36001)

In [ ]:
n10_blksplgapeg_wat

In [ ]:
n10plgapeg_lpwat

In [ ]:
n10plgapeg_lpwat[2]

In [ ]:
np.mean(n10plgapeg_lpwat[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_n10pp

In [ ]:
gg_n10plgapeg_wat = line_fit(np.mean(n10plgapeg_lpwat[2]),blen_n10pp)

In [ ]:
gg_n8plgapeg_n10wat = line_fit(np.mean(n8plgapeg_lpwat[2]),blen_n10pp)

In [ ]:
gg_n6plgapeg_n10wat = line_fit(np.mean(n6plgapeg_lpwat[2]),blen_n10pp)

In [ ]:
gg_n10plgapeg_wat

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 10 PLGA-PEG in water

In [ ]:
blppn10 = []
for i in range(n10_mon-1):
    if i < int((n10_mon/2)-1):
        # PLGA contour length values
        blppn10.append(COM_n10pp[0,0])    
    elif i == int((n10_mon/2)-1):
        #PLGA-PEG linker length
        blppn10.append(COM_n10pp[1,0]) 
    elif i > int((n10_mon/2)-1):
    #    # PEG contour length values
        blppn10.append(COM_n10pp[2,0]) 

# Contour Length, N = 6 PLGA-PEG in water
lc_ppn10 = np.sum(blppn10)
lc_ppn10

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegwat[1,2] = lc_ppn10
re2e_rgsq_plgapegwat

In [ ]:
a_ppn10 = np.mean(n10_blksplgapeg_wat['Avg Sq. end to end distance'])

In [ ]:
b_ppn10 = np.mean(n10_blksplgapeg_wat['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegwat[0,2] = a_ppn10/b_ppn10
re2e_rgsq_plgapegwat

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegwat[1,2] = lc_ppn10
rhy_rg_plgapegwat

In [ ]:
aa_ppn10 = np.mean(n10_blksplgapeg_wat['Avg Radius of gyration'])

In [ ]:
bb_ppn10 = np.mean(n10_blksplgapeg_wat['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegwat[0,2] = aa_ppn10/bb_ppn10
rhy_rg_plgapegwat

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegwat[0,2] = np.mean(n10_blksplgapeg_wat["Avg persistence length"])/lc_ppn10
kuhnlpL_plgapegwat

#### Block averaged Radius of gyration and persistence length, N = 10 PLGA-PEG in water

In [ ]:
np.mean(n10_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.std(n10_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.mean(n10_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
np.std(n10_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n6pp, np.log(cor_n6plgapeg_wat[0]), yerr=mk_n6ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n8pp, np.log(cor_n8plgapeg_wat[0]), yerr=mk_n8ppwat, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n10pp, np.log(cor_n10plgapeg_wat[0]), yerr=mk_n10ppwat, color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_n10pp, gg_n6plgapeg_n10wat, color='teal')
plt.plot(blen_n10pp, gg_n8plgapeg_n10wat, color='dodgerblue')
plt.plot(blen_n10pp, gg_n10plgapeg_wat, color='darkmagenta')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in water', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-6,1)
plt.xlim(0,40)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -5.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 19.4 $\AA$ ± 1.43 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -5.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 20.8 $\AA$ ± 1.03 $\AA$', fontsize=15, color='dodgerblue')
plt.text(0.5, -5.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 21.2 $\AA$ ± 0.58 $\AA$', fontsize=15, color='darkmagenta')

In [ ]:
rgplgapeg_olig_wat[r"$R_{g}$ [Angstrom] N = 10 PLGA-PEG water"] = n10_blksplgapeg_wat["Avg Radius of gyration"]
rgplgapeg_olig_wat

In [ ]:
pers_plgapegt_wat[r"$L_{p}$ [Angstrom] N = 10 PLGA-PEG water"] = n10_blksplgapeg_wat["Avg persistence length"]
pers_plgapegt_wat

### N = 16 PLGA-PEG/water 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n16_plgapeg_wat = mda.Universe("n16plga_peg_wat/n16plgapeg_WAT.pdb", "n16plga_peg_wat/NoWat_n16plgapeg.xtc")

In [ ]:
n16_plgapeg_wat.trajectory

In [ ]:
len(n16_plgapeg_wat.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n16wat = n16_plgapeg_wat.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n16wat.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n16plgapeg_wat = pers_length(plgapeg_n16wat,16)
crv_n16plgapeg_wat

#### Orientation Order parameter vs. time 

In [ ]:
n16_mon = 16 
start = 0
end = 36001

In [ ]:
OOp_n16plgapeg_wat = orientation_order_param(plgapeg_n16wat, n16_plgapeg_wat, n16_mon, start, end)

In [ ]:
OOp_n16plgapeg_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_wat,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_wat,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, OOp_n10plgapeg_wat,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, OOp_n16plgapeg_wat,linewidth=2, color='maroon')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 16 PLGA-PEG in water

In [ ]:
OOp_n16plgapeg_wat[82]

In [ ]:
OOp_n16plgapeg_wat[239]

In [ ]:
np.mean(OOp_n16plgapeg_wat)

In [ ]:
np.std(OOp_n16plgapeg_wat)

#### Mean squared end to end distance 

In [ ]:
eVec_n16plgapeg_wat, e2edis_n16plgapeg_wat = mean_sq_e2e(plgapeg_n16wat, n16_plgapeg_wat, n16_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_wat[1]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, e2edis_n8plgapeg_wat[1]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, e2edis_n10plgapeg_wat[1]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, e2edis_n16plgapeg_wat[1]/10,linewidth=2, color='maroon')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,7)

#### Ree Autocorrelation function

In [ ]:
e2edis_n16plgapeg_wat[1]

In [ ]:
t_corr = 7000
window_shift = 9

In [ ]:
ACn16plgapegwat_Ree, eqvar_n16plgapegwat = obs_autocorr_RA(e2edis_n16plgapeg_wat[1],t_corr, window_shift,start,end)

In [ ]:
ACn16plgapegwat_Ree

In [ ]:
eqvar_n16plgapegwat

In [ ]:
ACn16plgapegwat_sqRee, evsqRee_n16plgapegwat = obs_autocorr_RA(e2edis_n16plgapeg_wat[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n16plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Ree[1]/100, ACn6plgapegwat_Ree[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwat_Ree[1]/100, ACn8plgapegwat_Ree[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwat_Ree[1]/100, ACn10plgapegwat_Ree[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegwat_Ree[1]/100, ACn16plgapegwat_Ree[0],linewidth=2, color='purple')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree','N = 10 Ree','N = 16 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
#plt.grid(b=True)
plt.xlim(0,20)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn16wat_rh = hydro_rad_poly(plgapeg_n16wat, n16_plgapeg_wat, n16_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6wat_rh/10,linewidth=2, color='teal')
plt.plot(trj_len/100, plgapegn8wat_rh/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, plgapegn10wat_rh/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, plgapegn16wat_rh/10,linewidth=2, color='maroon')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,7)

#### Rh autocorrelation

In [ ]:
t_corr = 7000
window_shift = 9

In [ ]:
ACn16plgapegwatRh, eqvarRh_n16plgapegwat = obs_autocorr_RA(plgapegn16wat_rh,t_corr, window_shift,start,end)

In [ ]:
ACn16plgapegwatRh

In [ ]:
eqvarRh_n16plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegwatRh[1]/100, ACn6plgapegwatRh[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwatRh[1]/100, ACn8plgapegwatRh[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwatRh[1]/100, ACn10plgapegwatRh[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegwatRh[1]/100, ACn16plgapegwatRh[0],linewidth=2, color='purple')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in water','N = 8 in water','N = 10 in water','N = 16 in water'], frameon=False, fontsize=14)
plt.xlim(0,20)
#plt.ylim(-0.2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n16_mon = 16 
start = 0
end = 36001
t_corr = 2000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn16wat, tcSUM_plgapegn16wat = pos_bead_autocorr_RA(plgapeg_n16wat, n16_plgapeg_wat, n16_mon, t_corr, window_shift, start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn16wat

In [ ]:
tcSUM_plgapegn16wat.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn16wat = tcRA_plgapegn16wat[1]/100
ydata_plgapegn16wat = tcRA_plgapegn16wat[0]

In [ ]:
ydata_plgapegn16wat.shape

In [ ]:
xdata_plgapegn16wat.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, ydata_plgapegn10wat, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, ydata_plgapegn16wat, color='maroon', s=s_n16)
plt.title(r'Positional bead autocorrelation PLGA-PEG in Water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,100)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
ts = 1
hs = 0.1
n16_mon = 16
n_b = n16_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
rouse_relax(1, 2, 6)

In [ ]:
zimm_relax_func(1, 2, 0.1, 5)

In [ ]:
def res_plgapegn16wat(variabls, xnp, ynp):
    hs_np = variabls['h_star']
    tr1_np = variabls['t_first']
    n_bonds = 15
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n16wat = Parameters()
ppfit_n16wat.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n16wat.add(name='t_first', value=2)

In [ ]:
mini_ppn16wat = Minimizer(res_plgapegn16wat, ppfit_n16wat, fcn_args=(xdata_plgapegn16wat, ydata_plgapegn16wat))
out_ppn16wat = mini_ppn16wat.leastsq()
#bfit_n16 = ydata_n16wat + out_n16wat.residual 

In [ ]:
report_fit(out_ppn16wat.params)

In [ ]:
out_ppn16wat.params

In [ ]:
# with h varying, tr = 0.88882196 +/- 0.00497893,h* = 0.26000000 +/- 0.00391920
twat_n16plgapeg = []
n_bonds = 15
for i in range(len(xdata_plgapegn16wat)):
    twat_n16plgapeg.append(zimm_relax_fit(xdata_plgapegn16wat[i], 0.88882196, 0.26,  n_bonds))


In [ ]:
t_c, tz_n16plgapeg, zimm_n16plgapeg, rouse_n16plgapeg = zimm_relax_func(1, 0.88882196, 0.26,  n_bonds)

In [ ]:
len(tz_n16plgapeg)

In [ ]:
x_pos_n8 = np.arange(7)
x_pos_n10 = np.arange(9)
x_pos_n16 = np.arange(15)
width = 0.25
aa_types_n16 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$",
           r"$\tau_{8}$", r"$\tau_{9}$",r"$\tau_{10}$",r"$\tau_{11}$",r"$\tau_{12}$",r"$\tau_{13}$",r"$\tau_{14}$",r"$\tau_{15}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in water', fontsize=18)
plt.bar(x_pos_n8+(2*width), tz_n8plgapeg, width, ecolor='black',capsize=5, color='dodgerblue', label='N = 8')
plt.bar(x_pos_n10+(width), tz_n10plgapeg, width, ecolor='black',capsize=5, color='darkmagenta', label='N = 10')
plt.bar(x_pos_n16, tz_n16plgapeg, width, ecolor='black',capsize=5, color='maroon', label='N = 16')
plt.xticks(x_pos_n16, labels=aa_types_n16, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
aa_eigen_n16 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$",r"$a_{8}$",
           r"$a_{9}$",r"$a_{10}$",r"$a_{11}$",r"$a_{12}$",r"$a_{13}$",r"$a_{14}$",r"$a_{15}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in water N = 16', fontsize=18)
plt.bar(x_pos_n16, rouse_n16plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n16+width, zimm_n16plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n16+width,labels=aa_eigen_n16, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, ydata_plgapegn10wat, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, ydata_plgapegn16wat, color='maroon', s=s_n16)
plt.plot(xdata_plgapegn6wat, twat_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8wat, twat_n8plgapeg, color='dodgerblue')
plt.plot(xdata_plgapegn10wat, twat_n10plgapeg, color='darkmagenta')
plt.plot(xdata_plgapegn16wat, twat_n16plgapeg, color='maroon')
plt.title(r'Positional bead autocorrelation PLGA-PEG in water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,100)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppwat_msen16 = np.array([ydata_plgapegn16wat - np.array(twat_n16plgapeg)])
rppwat_msen16

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, rppwat_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, rppwat_msen8, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, rppwat_msen10, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, rppwat_msen16, color='maroon', s=s_n16)
plt.title(r'Relaxation time Fitting Residuals PLGA in water', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,100])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Radius of Gyration vs. time N = 16 PLGA-PEG in water

In [ ]:
# Key variables 
n16_mon = 16 
start = 0
end = 36001

In [ ]:
n16plgapeg_rgens_wat, cor_n16plgapeg_wat, n16plgapeg_cos_wat, rgwat_n16plgapeg = get_rg_pers_poly(plgapeg_n16wat, n16_plgapeg_wat,
                                                                                                  start,end)

In [ ]:
n16plgapeg_rgens_wat[0].shape

In [ ]:
cor_n16plgapeg_wat[3]

In [ ]:
n16plgapeg_cos_wat

In [ ]:
rgwat_n16plgapeg

In [ ]:
n16plgapeg_rgens_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_wat[0]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, n8plgapeg_rgens_wat[0]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, n10plgapeg_rgens_wat[0]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, n16plgapeg_rgens_wat[0]/10,linewidth=2, color='maroon')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,7)

#### Rg Autocorrelation

In [ ]:
ACn16plgapegwat_Rg, eqvarRg_n16plgapegwat = obs_autocorr_RA(n16plgapeg_rgens_wat[0],t_corr, window_shift,start,end)

In [ ]:
ACn16plgapegwat_Rg

In [ ]:
eqvarRg_n16plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in water autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Rg[1]/100, ACn6plgapegwat_Rg[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwat_Rg[1]/100, ACn8plgapegwat_Rg[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwat_Rg[1]/100, ACn10plgapegwat_Rg[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegwat_Rg[1]/100, ACn16plgapegwat_Rg[0],linewidth=2, color='purple')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg','N = 16 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

#### Get average COM distance, arch length values

In [ ]:
cb_n16plga = np.zeros(shape=(36001))
cb_n16pplinker = np.zeros(shape=(36001))
cb_n16peg = np.zeros(shape=(36001))
count = 0
for ts in n16_plgapeg_wat.trajectory[0:36001]:
    # 1st two PLGA residues
    n16m1_plga = n16_plgapeg_wat.select_atoms("resid 1")
    n16m2_plga = n16_plgapeg_wat.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n16pl_link = n16_plgapeg_wat.select_atoms("resid 8")
    n16peg_link = n16_plgapeg_wat.select_atoms("resid 9")
    # last two PEG residues 
    n16l1_peg = n16_plgapeg_wat.select_atoms("resid 15")
    n16l2_peg = n16_plgapeg_wat.select_atoms("resid 16")
    
    cb_n16plga[count] = mda.analysis.distances.distance_array(n16m1_plga.center_of_mass(), n16m2_plga.center_of_mass(), 
                                                   box=n16_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n16pplinker[count] = mda.analysis.distances.distance_array(n16pl_link.center_of_mass(), n16peg_link.center_of_mass(), 
                                                   box=n16_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n16peg[count] = mda.analysis.distances.distance_array(n16l1_peg.center_of_mass(), n16l2_peg.center_of_mass(), 
                                                   box=n16_plgapeg_wat.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_n16pp = np.zeros(shape=(3,2))
COM_n16pp[0,0] = np.mean(cb_n16plga)
COM_n16pp[1,0] = np.mean(cb_n16pplinker)
COM_n16pp[2,0] = np.mean(cb_n16peg)
COM_n16pp[0,1] = np.std(cb_n16plga)
COM_n16pp[1,1] = np.std(cb_n16pplinker)
COM_n16pp[2,1] = np.std(cb_n16peg)
COM_n16pp

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 16 PLGA-PEG/water

In [ ]:
# x values
blen_n16pp = np.zeros(shape=15)
for i in range(n16_mon-1):
    if i < int((n16_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_n16pp[i] = 0
        elif i != 0:
            blen_n16pp[i] = np.sum(blen_n16pp[i-1]) + COM_n16pp[0,0]    
    elif i == int((n16_mon/2)-1):
        #PLGA-PEG linker length
        blen_n16pp[i] = np.sum(blen_n16pp[i-1]) + COM_n16pp[1,0]
    elif i > int((n16_mon/2)-1):
    #    # PEG contour length values
        blen_n16pp[i] = np.sum(blen_n16pp[i-1]) + COM_n16pp[2,0]
        
blen_n16pp

In [ ]:
# Error prop. into natural log std deviation
mk_n16ppwat = cor_n16plgapeg_wat[1]/cor_n16plgapeg_wat[0]
mk_n16ppwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n16pp, np.log(cor_n16plgapeg_wat[0]), yerr=mk_n16ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in water
n16_blksplgapeg_wat , n16plgapeg_lpwat = bavg_pers_cnt(5, plgapeg_n16wat, n16_plgapeg_wat, blen_n16pp, 5, 0 , 36001)

In [ ]:
n16_blksplgapeg_wat

In [ ]:
n16plgapeg_lpwat

In [ ]:
n16plgapeg_lpwat[2]

In [ ]:
np.mean(n16plgapeg_lpwat[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_n16pp

In [ ]:
gg_n16plgapeg_wat = line_fit(np.mean(n16plgapeg_lpwat[2]),blen_n16pp)

In [ ]:
gg_n10plgapeg_n16wat = line_fit(np.mean(n10plgapeg_lpwat[2]),blen_n16pp)

In [ ]:
gg_n8plgapeg_n16wat = line_fit(np.mean(n8plgapeg_lpwat[2]),blen_n16pp)

In [ ]:
gg_n6plgapeg_n16wat = line_fit(np.mean(n6plgapeg_lpwat[2]),blen_n16pp)

In [ ]:
gg_n16plgapeg_wat

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 16 PLGA-PEG in water

In [ ]:
blppn16 = []
for i in range(n16_mon-1):
    if i < int((n16_mon/2)-1):
        # PLGA contour length values
        blppn16.append(COM_n16pp[0,0])    
    elif i == int((n16_mon/2)-1):
        #PLGA-PEG linker length
        blppn16.append(COM_n16pp[1,0]) 
    elif i > int((n16_mon/2)-1):
    #    # PEG contour length values
        blppn16.append(COM_n16pp[2,0]) 

# Contour Length, N = 6 PLGA-PEG in water
lc_ppn16 = np.sum(blppn16)
lc_ppn16

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegwat[1,3] = lc_ppn16
re2e_rgsq_plgapegwat

In [ ]:
a_ppn16 = np.mean(n16_blksplgapeg_wat['Avg Sq. end to end distance'])

In [ ]:
b_ppn16 = np.mean(n16_blksplgapeg_wat['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegwat[0,3] = a_ppn16/b_ppn16
re2e_rgsq_plgapegwat

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegwat[1,3] = lc_ppn16
rhy_rg_plgapegwat

In [ ]:
aa_ppn16 = np.mean(n16_blksplgapeg_wat['Avg Radius of gyration'])

In [ ]:
bb_ppn16 = np.mean(n16_blksplgapeg_wat['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegwat[0,3] = aa_ppn16/bb_ppn16
rhy_rg_plgapegwat

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegwat[0,3] = np.mean(n16_blksplgapeg_wat["Avg persistence length"])/lc_ppn16
kuhnlpL_plgapegwat

#### Block averaged Radius of gyration and persistence length, N = 16 PLGA-PEG in water

In [ ]:
np.mean(n16_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.std(n16_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.mean(n16_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
np.std(n16_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n6pp, np.log(cor_n6plgapeg_wat[0]), yerr=mk_n6ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n8pp, np.log(cor_n8plgapeg_wat[0]), yerr=mk_n8ppwat, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n10pp, np.log(cor_n10plgapeg_wat[0]), yerr=mk_n10ppwat, color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n16pp, np.log(cor_n16plgapeg_wat[0]), yerr=mk_n16ppwat, color='maroon', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_n16pp, gg_n6plgapeg_n16wat, color='teal')
plt.plot(blen_n16pp, gg_n8plgapeg_n16wat, color='dodgerblue')
plt.plot(blen_n16pp, gg_n10plgapeg_n16wat, color='darkmagenta')
plt.plot(blen_n16pp, gg_n16plgapeg_wat, color='maroon')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in water', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-6,1)
plt.xlim(0,70)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -5.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 19.4 $\AA$ ± 1.43 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -5.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 20.8 $\AA$ ± 1.03 $\AA$', fontsize=15, color='dodgerblue')
plt.text(0.5, -5.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 21.2 $\AA$ ± 0.58 $\AA$', fontsize=15, color='darkmagenta')
plt.text(0.5, -4.90,r'$N_{PLGA-PEG}$ = 16: $L_{p}$ = 23.1 $\AA$ ± 0.86 $\AA$', fontsize=15, color='maroon')

In [ ]:
rgplgapeg_olig_wat[r"$R_{g}$ [Angstrom] N = 16 PLGA-PEG water"] = n16_blksplgapeg_wat["Avg Radius of gyration"]
rgplgapeg_olig_wat

In [ ]:
pers_plgapegt_wat[r"$L_{p}$ [Angstrom] N = 16 PLGA-PEG water"] = n16_blksplgapeg_wat["Avg persistence length"]
pers_plgapegt_wat

### N = 20 PLGA-PEG/water 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n20_plgapeg_wat = mda.Universe("n20plga_peg_wat/n20plgapeg_WAT.pdb", "n20plga_peg_wat/NoWat_n20plgapeg.xtc")

In [ ]:
n20_plgapeg_wat.trajectory

In [ ]:
len(n20_plgapeg_wat.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n20wat = n20_plgapeg_wat.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n20wat.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n20plgapeg_wat = pers_length(plgapeg_n20wat,20)
crv_n20plgapeg_wat

#### Orientation Order parameter vs. time 

In [ ]:
n20_mon = 20
start = 0
end = 36001

In [ ]:
OOp_n20plgapeg_wat = orientation_order_param(plgapeg_n20wat, n20_plgapeg_wat, n20_mon, start, end)

In [ ]:
OOp_n20plgapeg_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_wat,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_wat,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, OOp_n10plgapeg_wat,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, OOp_n16plgapeg_wat,linewidth=2, color='maroon')
plt.plot(trj_len/100, OOp_n20plgapeg_wat,linewidth=2, color='orangered')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water','N = 20 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-1,1)

#### Mean Orientation Order Parameter for N = 20 PLGA-PEG in water

In [ ]:
OOp_n20plgapeg_wat[82]

In [ ]:
OOp_n20plgapeg_wat[239]

In [ ]:
np.mean(OOp_n20plgapeg_wat)

In [ ]:
np.std(OOp_n20plgapeg_wat)

#### Mean squared end to end distance 

In [ ]:
eVec_n20plgapeg_wat, e2edis_n20plgapeg_wat = mean_sq_e2e(plgapeg_n20wat, n20_plgapeg_wat, n20_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
plt.plot(trj_len/100, e2edis_n6plgapeg_wat[1]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, e2edis_n8plgapeg_wat[1]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, e2edis_n10plgapeg_wat[1]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, e2edis_n16plgapeg_wat[1]/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, e2edis_n20plgapeg_wat[1]/10,linewidth=2, color='orangered')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water','N = 20 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,7)

#### Ree Autocorrelation function

In [ ]:
e2edis_n20plgapeg_wat[1]

In [ ]:
t_corr = 4000
window_shift = 6

In [ ]:
ACn20plgapegwat_Ree, eqvar_n120pegwat = obs_autocorr_RA(e2edis_n20plgapeg_wat[1],t_corr, window_shift,start,end)

In [ ]:
ACn20plgapegwat_Ree

In [ ]:
eqvar_n16plgapegwat

In [ ]:
ACn20plgapegwat_sqRee, evsqRee_n20plgapegwat = obs_autocorr_RA(e2edis_n20plgapeg_wat[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n20plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG Ree autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Ree[1]/100, ACn6plgapegwat_Ree[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwat_Ree[1]/100, ACn8plgapegwat_Ree[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwat_Ree[1]/100, ACn10plgapegwat_Ree[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegwat_Ree[1]/100, ACn16plgapegwat_Ree[0],linewidth=2, color='purple')
plt.plot(ACn20plgapegwat_Ree[1]/100, ACn20plgapegwat_Ree[0],linewidth=2, color='#8B7F47')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree','N = 10 Ree','N = 15 Ree','N = 20 Ree'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn20wat_rh = hydro_rad_poly(plgapeg_n20wat, n20_plgapeg_wat, n20_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
plt.plot(trj_len/100, plgapegn6wat_rh/10,linewidth=2, color='teal')
plt.plot(trj_len/100, plgapegn8wat_rh/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, plgapegn10wat_rh/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, plgapegn16wat_rh/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, plgapegn20wat_rh/10,linewidth=2, color='orangered')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water','N = 20 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,6)

#### Rh autocorrelation

In [ ]:
ACn20plgapegwatRh, eqvarRh_n20plgapegwat = obs_autocorr_RA(plgapegn20wat_rh,t_corr, window_shift,start,end)

In [ ]:
ACn20plgapegwatRh

In [ ]:
eqvarRh_n20plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Rh autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{Rh}(t)$', fontsize=15)
plt.plot(ACn6plgapegwatRh[1]/100, ACn6plgapegwatRh[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwatRh[1]/100, ACn8plgapegwatRh[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwatRh[1]/100, ACn10plgapegwatRh[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegwatRh[1]/100, ACn16plgapegwatRh[0],linewidth=2, color='purple')
plt.plot(ACn20plgapegwatRh[1]/100, ACn20plgapegwatRh[0],linewidth=2, color='#8B7F47')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in water','N = 8 in water','N = 10 in water','N = 15 in water','N = 20 in water'], frameon=False, fontsize=14)
plt.xlim(0,20)
#plt.ylim(-2,1.5)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n20_mon = 20 
start = 0
end = 36001
t_corr = 2000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn20wat, tcSUM_plgapegn20wat = pos_bead_autocorr_RA(plgapeg_n20wat, n20_plgapeg_wat, n20_mon, t_corr, window_shift, start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn20wat

In [ ]:
tcSUM_plgapegn20wat.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn20wat = tcRA_plgapegn20wat[1]/100
ydata_plgapegn20wat = tcRA_plgapegn20wat[0]

In [ ]:
ydata_plgapegn20wat.shape

In [ ]:
xdata_plgapegn20wat.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, ydata_plgapegn10wat, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, ydata_plgapegn16wat, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20wat, ydata_plgapegn20wat, color='orangered', s=s_n20)
plt.title(r'Positional bead autocorrelation PLGA-PEG in Water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
ts = 1
hs = 0.1
n20_mon = 20
n_b = n20_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
zimm_relax_func(ts,tr1,hs,n_b)

In [ ]:
def res_plgapegn20wat(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 19
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n20wat = Parameters()
#ppfit_n20wat.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n20wat.add(name='t_first', value=2)

In [ ]:
mini_ppn20wat = Minimizer(res_plgapegn20wat, ppfit_n20wat, fcn_args=(xdata_plgapegn20wat, ydata_plgapegn20wat))
out_ppn20wat = mini_ppn20wat.leastsq()
#bfit_n10 = ydata_n10wat + out_n10wat.residual 

In [ ]:
report_fit(out_ppn20wat.params)

In [ ]:
out_ppn20wat.params

In [ ]:
# with h varying, tr = 1.13052860 +/- 0.00845211,h* = 0.25999989 +/- 0.05636451
twat_n20plgapeg = []
n_bonds = 19
for i in range(len(xdata_plgapegn20wat)):
    twat_n20plgapeg.append(zimm_relax_fit(xdata_plgapegn20wat[i], 1.13179195, 0.26,  n_bonds))


In [ ]:
t_c, tz_n20plgapeg, zimm_n20plgapeg, rouse_n20plgapeg = zimm_relax_func(1, 1.13179195, 0.26,  n_bonds)

In [ ]:
x_pos_n10 = np.arange(9)
x_pos_n16 = np.arange(15)
x_pos_n20 = np.arange(19)
width = 0.25
aa_types_n20 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$",
           r"$\tau_{8}$", r"$\tau_{9}$",r"$\tau_{10}$", r"$\tau_{11}$", r"$\tau_{12}$", r"$\tau_{13}$", r"$\tau_{14}$",
            r"$\tau_{15}$",r"$\tau_{16}$",r"$\tau_{17}$", r"$\tau_{18}$", r"$\tau_{19}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in water', fontsize=18)
plt.bar(x_pos_n10+(2*width), tz_n10plgapeg, width, ecolor='black',capsize=5, color='darkmagenta', label='N = 10', align='center')
plt.bar(x_pos_n16+width, tz_n16plgapeg, width, ecolor='black',capsize=5, color='maroon', label='N = 16', align='center')
plt.bar(x_pos_n20, tz_n20plgapeg, width, ecolor='black',capsize=5, color='orangered', label='N = 20', align='center')
plt.xticks(x_pos_n20, labels=aa_types_n20, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
aa_eigen_n20 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$",r"$a_{8}$",
           r"$a_{9}$",r"$a_{10}$", r"$a_{11}$", r"$a_{12}$", r"$a_{13}$", r"$a_{14}$",r"$a_{15}$",r"$a_{16}$",r"$a_{17}$",
           r"$a_{18}$",r"$a_{19}$"]
fig = plt.figure(figsize=(7,7))
plt.title('PLGA-PEG in water N = 20', fontsize=18)
plt.bar(x_pos_n20, rouse_n20plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n20+width, zimm_n20plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n20+width,labels=aa_eigen_n20, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, ydata_plgapegn10wat, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, ydata_plgapegn16wat, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20wat, ydata_plgapegn20wat, color='orangered', s=s_n20)
plt.plot(xdata_plgapegn6wat, twat_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8wat, twat_n8plgapeg, color='dodgerblue')
plt.plot(xdata_plgapegn10wat, twat_n10plgapeg, color='darkmagenta')
plt.plot(xdata_plgapegn16wat, twat_n16plgapeg, color='maroon')
plt.plot(xdata_plgapegn20wat, twat_n20plgapeg, color='orangered')
plt.title(r'Positional bead autocorrelation PLGA-PEG in water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,100)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppwat_msen20 = np.array([ydata_plgapegn20wat - np.array(twat_n20plgapeg)])
rppwat_msen20

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, rppwat_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, rppwat_msen8, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, rppwat_msen10, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, rppwat_msen16, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20wat, rppwat_msen20, color='orangered', s=s_n20)
plt.title(r'Relaxation time Fitting Residuals PLGA in water', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,100])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Radius of Gyration vs. time N = 20 PLGA-PEG in water

In [ ]:
# Key variables 
n20_mon = 20
start = 0
end = 36001

In [ ]:
n20plgapeg_rgens_wat, cor_n20plgapeg_wat, n20plgapeg_cos_wat, rgwat_n20plgapeg = get_rg_pers_poly(plgapeg_n20wat, n20_plgapeg_wat,
                                                                                                  start,end)

In [ ]:
n20plgapeg_rgens_wat[0].shape

In [ ]:
cor_n20plgapeg_wat[3]

In [ ]:
n20plgapeg_cos_wat

In [ ]:
rgwat_n20plgapeg

In [ ]:
n20plgapeg_rgens_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
plt.plot(trj_len/100, n6plgapeg_rgens_wat[0]/10,linewidth=2, color='teal')
plt.plot(trj_len/100, n8plgapeg_rgens_wat[0]/10,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, n10plgapeg_rgens_wat[0]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, n16plgapeg_rgens_wat[0]/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, n20plgapeg_rgens_wat[0]/10,linewidth=2, color='orangered')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water','N = 20 in water'], frameon=False, fontsize=14)
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,2)

#### Rg Autocorrelation

In [ ]:
ACn20plgapegwat_Rg, eqvarRg_n20plgapegwat = obs_autocorr_RA(n20plgapeg_rgens_wat[0],t_corr, window_shift,start,end)

In [ ]:
ACn20plgapegwat_Rg

In [ ]:
eqvarRg_n20plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PEG in water autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Rg[1]/100, ACn6plgapegwat_Rg[0],linewidth=2, color='#1F2E69')
plt.plot(ACn8plgapegwat_Rg[1]/100, ACn8plgapegwat_Rg[0],linewidth=2, color='#4C80ED')
plt.plot(ACn10plgapegwat_Rg[1]/100, ACn10plgapegwat_Rg[0],linewidth=2, color='#8C52FC')
plt.plot(ACn16plgapegwat_Rg[1]/100, ACn16plgapegwat_Rg[0],linewidth=2, color='purple')
plt.plot(ACn20plgapegwat_Rg[1]/100, ACn20plgapegwat_Rg[0],linewidth=2, color='#8B7F47')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg','N = 15 Rg','N = 20 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

#### Get average COM distance, arch length values

In [ ]:
cb_n20plga = np.zeros(shape=(36001))
cb_n20pplinker = np.zeros(shape=(36001))
cb_n20peg = np.zeros(shape=(36001))
count = 0
for ts in n20_plgapeg_wat.trajectory[0:36001]:
    # 1st two PLGA residues
    n20m1_plga = n20_plgapeg_wat.select_atoms("resid 1")
    n20m2_plga = n20_plgapeg_wat.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n20pl_link = n20_plgapeg_wat.select_atoms("resid 10")
    n20peg_link = n20_plgapeg_wat.select_atoms("resid 11")
    # last two PEG residues 
    n20l1_peg = n20_plgapeg_wat.select_atoms("resid 19")
    n20l2_peg = n20_plgapeg_wat.select_atoms("resid 20")
    
    cb_n20plga[count] = mda.analysis.distances.distance_array(n20m1_plga.center_of_mass(), n20m2_plga.center_of_mass(), 
                                                   box=n20_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n20pplinker[count] = mda.analysis.distances.distance_array(n20pl_link.center_of_mass(), n20peg_link.center_of_mass(), 
                                                   box=n20_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n20peg[count] = mda.analysis.distances.distance_array(n20l1_peg.center_of_mass(), n20l2_peg.center_of_mass(), 
                                                   box=n20_plgapeg_wat.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_n20pp = np.zeros(shape=(3,2))
COM_n20pp[0,0] = np.mean(cb_n20plga)
COM_n20pp[1,0] = np.mean(cb_n20pplinker)
COM_n20pp[2,0] = np.mean(cb_n20peg)
COM_n20pp[0,1] = np.std(cb_n20plga)
COM_n20pp[1,1] = np.std(cb_n20pplinker)
COM_n20pp[2,1] = np.std(cb_n20peg)
COM_n20pp

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 20 PLGA-PEG/water

In [ ]:
# x values
blen_n20pp = np.zeros(shape=19)
for i in range(n20_mon-1):
    if i < int((n20_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_n20pp[i] = 0
        elif i != 0:
            blen_n20pp[i] = np.sum(blen_n20pp[i-1]) + COM_n20pp[0,0]    
    elif i == int((n20_mon/2)-1):
        #PLGA-PEG linker length
        blen_n20pp[i] = np.sum(blen_n20pp[i-1]) + COM_n20pp[1,0]
    elif i > int((n20_mon/2)-1):
    #    # PEG contour length values
        blen_n20pp[i] = np.sum(blen_n20pp[i-1]) + COM_n20pp[2,0]
        
blen_n20pp

In [ ]:
# Error prop. into natural log std deviation
mk_n20ppwat = cor_n20plgapeg_wat[1]/cor_n20plgapeg_wat[0]
mk_n20ppwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n20pp, np.log(cor_n20plgapeg_wat[0]), yerr=mk_n20ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in water
n20_blksplgapeg_wat , n20plgapeg_lpwat = bavg_pers_cnt(5, plgapeg_n20wat, n20_plgapeg_wat, blen_n20pp, 5, 0 , 36001)

In [ ]:
n20_blksplgapeg_wat

In [ ]:
n20plgapeg_lpwat

In [ ]:
n20plgapeg_lpwat[2]

In [ ]:
np.mean(n20plgapeg_lpwat[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_n20pp

In [ ]:
gg_n20plgapeg_wat = line_fit(np.mean(n20plgapeg_lpwat[2]),blen_n20pp)

In [ ]:
gg_n16plgapeg_n20wat = line_fit(np.mean(n16plgapeg_lpwat[2]),blen_n20pp)

In [ ]:
gg_n10plgapeg_n20wat = line_fit(np.mean(n10plgapeg_lpwat[2]),blen_n20pp)

In [ ]:
gg_n8plgapeg_n20wat = line_fit(np.mean(n8plgapeg_lpwat[2]),blen_n20pp)

In [ ]:
gg_n6plgapeg_n20wat = line_fit(np.mean(n6plgapeg_lpwat[2]),blen_n20pp)

In [ ]:
gg_n20plgapeg_wat

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 20 PLGA-PEG in water

In [ ]:
blppn20 = []
for i in range(n20_mon-1):
    if i < int((n20_mon/2)-1):
        # PLGA contour length values
        blppn20.append(COM_n20pp[0,0])    
    elif i == int((n20_mon/2)-1):
        #PLGA-PEG linker length
        blppn20.append(COM_n20pp[1,0]) 
    elif i > int((n20_mon/2)-1):
    #    # PEG contour length values
        blppn20.append(COM_n20pp[2,0]) 

# Contour Length, N = 6 PLGA-PEG in water
lc_ppn20 = np.sum(blppn20)
lc_ppn20

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegwat[1,4] = lc_ppn20
re2e_rgsq_plgapegwat

In [ ]:
a_ppn20 = np.mean(n20_blksplgapeg_wat['Avg Sq. end to end distance'])

In [ ]:
b_ppn20 = np.mean(n20_blksplgapeg_wat['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegwat[0,4] = a_ppn20/b_ppn20
re2e_rgsq_plgapegwat

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegwat[1,4] = lc_ppn20
rhy_rg_plgapegwat

In [ ]:
aa_ppn20 = np.mean(n20_blksplgapeg_wat['Avg Radius of gyration'])

In [ ]:
bb_ppn20 = np.mean(n20_blksplgapeg_wat['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegwat[0,4] = aa_ppn20/bb_ppn20
rhy_rg_plgapegwat

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegwat[0,4] = np.mean(n20_blksplgapeg_wat["Avg persistence length"])/lc_ppn20
kuhnlpL_plgapegwat

#### Block averaged Radius of gyration and persistence length, N = 20 PLGA-PEG in water

In [ ]:
np.mean(n20_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.std(n20_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.mean(n20_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
np.std(n20_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n6pp, np.log(cor_n6plgapeg_wat[0]), yerr=mk_n6ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n8pp, np.log(cor_n8plgapeg_wat[0]), yerr=mk_n8ppwat, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n10pp, np.log(cor_n10plgapeg_wat[0]), yerr=mk_n10ppwat, color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n16pp, np.log(cor_n16plgapeg_wat[0]), yerr=mk_n16ppwat, color='maroon', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n20pp, np.log(cor_n20plgapeg_wat[0]), yerr=mk_n20ppwat, color='orangered', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_n20pp, gg_n6plgapeg_n20wat, color='teal')
plt.plot(blen_n20pp, gg_n8plgapeg_n20wat, color='dodgerblue')
plt.plot(blen_n20pp, gg_n10plgapeg_n20wat, color='darkmagenta')
plt.plot(blen_n20pp, gg_n16plgapeg_n20wat, color='orangered')
plt.plot(blen_n20pp, gg_n20plgapeg_wat, color='maroon')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in water', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-7,1.5)
plt.xlim(0,90)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -6.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 19.4 $\AA$ ± 1.43 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -6.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 20.8 $\AA$ ± 1.03 $\AA$', fontsize=15, color='dodgerblue')
plt.text(0.5, -6.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 21.2 $\AA$ ± 0.58 $\AA$', fontsize=15, color='darkmagenta')
plt.text(0.5, -5.90,r'$N_{PLGA-PEG}$ = 16: $L_{p}$ = 23.1 $\AA$ ± 0.86 $\AA$', fontsize=15, color='orangered')
plt.text(0.5, -5.50,r'$N_{PLGA-PEG}$ = 20: $L_{p}$ = 23.2 $\AA$ ± 0.60 $\AA$', fontsize=15, color='maroon')

In [ ]:
rgplgapeg_olig_wat[r"$R_{g}$ [Angstrom] N = 20 PLGA-PEG water"] = n20_blksplgapeg_wat["Avg Radius of gyration"]
rgplgapeg_olig_wat

In [ ]:
pers_plgapegt_wat[r"$L_{p}$ [Angstrom] N = 20 PLGA-PEG water"] = n20_blksplgapeg_wat["Avg persistence length"]
pers_plgapegt_wat

### N = 30 PLGA-PEG/water 

In [ ]:
# For the right Rg calculation using MD Analysis, use trajactory without pbc 
n30_plgapeg_wat = mda.Universe("n30plga_peg_wat/n30plgapeg_WAT.pdb", "n30plga_peg_wat/NoWat_n30plgapeg.xtc")

In [ ]:
n30_plgapeg_wat.trajectory

In [ ]:
len(n30_plgapeg_wat.trajectory)

In [ ]:
#Select the polymer heavy atoms 
plgapeg_n30wat = n30_plgapeg_wat.select_atoms("resname sPLG PLG bPLG bPEG PEG sPEG and not type H")

In [ ]:
np.unique(plgapeg_n30wat.resids)

In [ ]:
# Get cosine theta values at frame 0 
crv_n30plgapeg_wat = pers_length(plgapeg_n30wat,30)
crv_n30plgapeg_wat

#### Orientation Order parameter vs. time 

In [ ]:
n30_mon = 30
start = 0
end = 36001

In [ ]:
OOp_n30plgapeg_wat = orientation_order_param(plgapeg_n30wat, n30_plgapeg_wat, n30_mon, start, end)

In [ ]:
OOp_n30plgapeg_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Orientation Order Parameter', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'Z', fontsize=15)
plt.plot(trj_len/100, OOp_n6plgapeg_wat,linewidth=2, color='teal')
plt.plot(trj_len/100, OOp_n8plgapeg_wat,linewidth=2, color='dodgerblue')
plt.plot(trj_len/100, OOp_n10plgapeg_wat,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, OOp_n16plgapeg_wat,linewidth=2, color='maroon')
plt.plot(trj_len/100, OOp_n20plgapeg_wat,linewidth=2, color='orangered')
plt.plot(trj_len/100, OOp_n30plgapeg_wat,linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water','N = 20 in water','N = 30 in water'],
           frameon=False, fontsize=14, loc='upper left')
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(-2,2)

#### Mean Orientation Order Parameter for N = 30 PLGA-PEG in water

In [ ]:
OOp_n30plgapeg_wat[82]

In [ ]:
OOp_n30plgapeg_wat[239]

In [ ]:
np.mean(OOp_n30plgapeg_wat)

In [ ]:
np.std(OOp_n30plgapeg_wat)

#### Mean squared end to end distance 

In [ ]:
eVec_n30plgapeg_wat, e2edis_n30plgapeg_wat = mean_sq_e2e(plgapeg_n30wat, n30_plgapeg_wat, n30_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG End to End distance', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{ee}$ [nm]', fontsize=15)
#plt.plot(trj_len/100, e2edis_n6plgapeg_wat[1]/10,linewidth=2, color='teal')
#plt.plot(trj_len/100, e2edis_n8plgapeg_wat[1]/10,linewidth=2, color='dodgerblue')
#plt.plot(trj_len/100, e2edis_n10plgapeg_wat[1]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, e2edis_n16plgapeg_wat[1]/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, e2edis_n20plgapeg_wat[1]/10,linewidth=2, color='orangered')
plt.plot(trj_len/100, e2edis_n30plgapeg_wat[1]/10,linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
#plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water'],frameon=False, fontsize=14, loc='upper left')
plt.legend(['N = 16 in Water', 'N = 20 in Water','N = 30 in Water'],frameon=False, fontsize=14, loc='upper left')
#plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water',
#            'N = 16 in water','N = 20 in water','N = 30 in water'],
#           frameon=False, fontsize=14, loc='upper left')
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0,10)

In [ ]:
np.save('n6plgapeg_watLEE.npy', e2edis_n6plgapeg_wat)
np.save('n8plgapeg_watLEE.npy', e2edis_n8plgapeg_wat)
np.save('n10plgapeg_watLEE.npy', e2edis_n10plgapeg_wat)
np.save('n16plgapeg_watLEE.npy', e2edis_n16plgapeg_wat)
np.save('n20plgapeg_watLEE.npy', e2edis_n20plgapeg_wat)
np.save('n30plgapeg_watLEE.npy', e2edis_n30plgapeg_wat)

In [ ]:
e2edis_n6plgapeg_wat = np.load('n6plgapeg_watLEE.npy', allow_pickle=True)
e2edis_n8plgapeg_wat = np.load('n8plgapeg_watLEE.npy', allow_pickle=True)
e2edis_n10plgapeg_wat = np.load('n10plgapeg_watLEE.npy', allow_pickle=True)
e2edis_n16plgapeg_wat = np.load('n16plgapeg_watLEE.npy', allow_pickle=True)
e2edis_n20plgapeg_wat = np.load('n20plgapeg_watLEE.npy', allow_pickle=True)
e2edis_n30plgapeg_wat = np.load('n30plgapeg_watLEE.npy', allow_pickle=True)

#### Ree Autocorrelation function

In [ ]:
e2edis_n30plgapeg_wat[1]

In [ ]:
t_corr = 3000
window_shift = 5

In [ ]:
ACn30plgapegwat_Ree, eqvar_n30plgapegwat = obs_autocorr_RA(e2edis_n30plgapeg_wat[1],t_corr, window_shift,start,end)

In [ ]:
ACn30plgapegwat_Ree

In [ ]:
eqvar_n30plgapegwat

In [ ]:
ACn30plgapegwat_sqRee, evsqRee_n30plgapegwat = obs_autocorr_RA(e2edis_n30plgapeg_wat[0],t_corr, window_shift,start,end)

In [ ]:
np.sqrt(evsqRee_n30plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in Water autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Ree[1]/100, ACn6plgapegwat_Ree[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegwat_Ree[1]/100, ACn8plgapegwat_Ree[0],linewidth=2, color='dodgerblue')
plt.plot(ACn10plgapegwat_Ree[1]/100, ACn10plgapegwat_Ree[0],linewidth=2, color='darkmagenta')
plt.plot(ACn16plgapegwat_Ree[1]/100, ACn16plgapegwat_Ree[0],linewidth=2, color='maroon')
plt.plot(ACn20plgapegwat_Ree[1]/100, ACn20plgapegwat_Ree[0],linewidth=2, color='orangered')
plt.plot(ACn30plgapegwat_Ree[1]/100, ACn30plgapegwat_Ree[0],linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Ree','N = 8 Ree','N = 10 Ree','N = 16 Ree','N = 20 Ree','N = 30 Ree'], frameon=False, fontsize=14)
#plt.legend(['N = 6 Lee','N = 8 Lee','N = 10 Lee','N = 15 Lee'], frameon=False, fontsize=14)
#plt.legend(['N = 20 Lee','N = 30 Lee'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

### Hydrodynamic Radius vs time

In [ ]:
plgapegn30wat_rh = hydro_rad_poly(plgapeg_n30wat, n30_plgapeg_wat, n30_mon, start, end)

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Hydrodynamic Radius', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{hyd}$ [nm]', fontsize=15)
#plt.plot(trj_len/100, plgapegn6wat_rh/10,linewidth=2, color='teal')
#plt.plot(trj_len/100, plgapegn8wat_rh/10,linewidth=2, color='dodgerblue')
#plt.plot(trj_len/100, plgapegn10wat_rh/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, plgapegn16wat_rh/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, plgapegn20wat_rh/10,linewidth=2, color='orangered')
plt.plot(trj_len/100, plgapegn30wat_rh/10,linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
#plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water'],frameon=False, fontsize=14, loc='upper left')
plt.legend(['N = 16 in Water', 'N = 20 in Water','N = 30 in Water'],frameon=False, fontsize=14, loc='upper left')
#plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water','N = 20 in water','N = 30 in water'],
#           frameon=False, fontsize=14, loc='upper left')
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(1,4)

In [ ]:
np.save('n6plgapeg_watRHYD.npy', plgapegn6wat_rh)
np.save('n8plgapeg_watRHYD.npy', plgapegn8wat_rh)
np.save('n10plgapeg_watRHYD.npy', plgapegn10wat_rh)
np.save('n16plgapeg_watRHYD.npy', plgapegn16wat_rh)
np.save('n20plgapeg_watRHYD.npy', plgapegn20wat_rh)
np.save('n30plgapeg_watRHYD.npy', plgapegn30wat_rh)

In [ ]:
plgapegn6wat_rh = np.load('n6plgapeg_watRHYD.npy', allow_pickle=True)
plgapegn8wat_rh = np.load('n8plgapeg_watRHYD.npy', allow_pickle=True)
plgapegn10wat_rh = np.load('n10plgapeg_watRHYD.npy', allow_pickle=True)
plgapegn16wat_rh = np.load('n16plgapeg_watRHYD.npy', allow_pickle=True)
plgapegn20wat_rh = np.load('n20plgapeg_watRHYD.npy', allow_pickle=True)
plgapegn30wat_rh = np.load('n30plgapeg_watRHYD.npy', allow_pickle=True)

#### Rh autocorrelation

In [ ]:
ACn30plgapegwatRh, eqvarRh_n30plgapegwat = obs_autocorr_RA(plgapegn30wat_rh,t_corr, window_shift,start,end)

In [ ]:
ACn30plgapegwatRh

In [ ]:
eqvarRh_n30plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in water autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwatRh[1]/100, ACn6plgapegwatRh[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegwatRh[1]/100, ACn8plgapegwatRh[0],linewidth=2, color='dodgerblue')
plt.plot(ACn10plgapegwatRh[1]/100, ACn10plgapegwatRh[0],linewidth=2, color='darkmagenta')
plt.plot(ACn16plgapegwatRh[1]/100, ACn16plgapegwatRh[0],linewidth=2, color='maroon')
plt.plot(ACn20plgapegwatRh[1]/100, ACn20plgapegwatRh[0],linewidth=2, color='orangered')
plt.plot(ACn30plgapegwatRh[1]/100, ACn30plgapegwatRh[0],linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
plt.legend([r'N = 6 $R_{hyd}$','N = 8 $R_{hyd}$','N = 10 $R_{hyd}$','N = 16 $R_{hyd}$','N = 20 $R_{hyd}$','N = 30 $R_{hyd}$']
 ,frameon=False, fontsize=14)
#plt.legend([r'N = 6 $R_{hyd}$','N = 8 $R_{hyd}$','N = 10 $R_{hyd}$','N = 15 $R_{hyd}$'], frameon=False, fontsize=14)
#plt.legend([r'N = 20 $R_{hyd}$',r'N = 30 $R_{hyd}$'], frameon=False, fontsize=14)
plt.xlim(0,20)
plt.ylim(-2,1)

### Relaxation times vs monomer length 

In [ ]:
# Key variables 
# def pos_bead_autocorr_RA(polymer_atoms, universe, n_monomers, t_corr,  start, end):
n30_mon = 30 
start = 0
end = 36001
t_corr = 2000
window_shift = 20

In [ ]:
s_time = timeit.default_timer()
tcRA_plgapegn30wat, tcSUM_plgapegn30wat = pos_bead_autocorr_RA(plgapeg_n30wat, n30_plgapeg_wat, n30_mon, t_corr, window_shift, start, end)
timeit.default_timer() - s_time

In [ ]:
tcRA_plgapegn30wat

In [ ]:
tcSUM_plgapegn30wat.shape

### Fitting autocorrelation data 

In [ ]:
xdata_plgapegn30wat = tcRA_plgapegn30wat[1]/100
ydata_plgapegn30wat = tcRA_plgapegn30wat[0]

In [ ]:
ydata_plgapegn30wat.shape

In [ ]:
xdata_plgapegn30wat.shape

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, ydata_plgapegn10wat, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, ydata_plgapegn16wat, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20wat, ydata_plgapegn20wat, color='orangered', s=s_n20)
plt.scatter(xdata_plgapegn30wat, ydata_plgapegn30wat, color='darkolivegreen', s=s_n30)
plt.title(r'Positional bead autocorrelation PLGA-PEG in Water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,300)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
np.save('n6plgapeg_watCt_RA.npy', tcRA_plgapegn6wat)
np.save('n8plgapeg_watCt_RA.npy', tcRA_plgapegn8wat)
np.save('n10plgapeg_watCt_RA.npy', tcRA_plgapegn10wat)
np.save('n10plgapeg_watCt_RA.npy', tcRA_plgapegn16wat)
np.save('n20plgapeg_watCt_RA.npy', tcRA_plgapegn20wat)
np.save('n30plgapeg_watCt_RA.npy', tcRA_plgapegn30wat)

In [ ]:
ts = 1
hs = 0.1
n30_mon = 30
n_b = n30_mon - 1
tr1 = 2
zimm_relax_fit(ts,tr1,hs,n_b)

In [ ]:
#zimm_relax_func(ts,tr1,hs,n_b)

In [ ]:
def res_plgapegn30wat(variabls, xnp, ynp):
    #hs_np = variabls['h_star']
    hs_np = 0.26
    tr1_np = variabls['t_first']
    n_bonds = 29
    testnp = []
    for i in range(len(xnp)):
        model_ynp = zimm_relax_fit(xnp[i], tr1_np, hs_np, n_bonds)
        #model_ynp = rouse_relax(xnp[i], tr1_np, n_m)
        testnp.append(ynp[i] - model_ynp)
    
    tt_n6 = np.array(testnp)

    return tt_n6
    
#x1 = np.array([0,0])
#pfit, pcov, infodict, errmsg, success = leastsq(res_poly, x1, args=(xdata, ydata), full_output=1)

In [ ]:
ppfit_n30wat = Parameters()
#ppfit_n30wat.add(name='h_star', value=0, min=0, max=0.26, vary=True)
ppfit_n30wat.add(name='t_first', value=2)

In [ ]:
mini_ppn30wat = Minimizer(res_plgapegn30wat, ppfit_n30wat, fcn_args=(xdata_plgapegn30wat, ydata_plgapegn30wat))
out_ppn30wat = mini_ppn30wat.leastsq()
#bfit_n10 = ydata_n10wat + out_n10wat.residual 

In [ ]:
report_fit(out_ppn30wat.params)

In [ ]:
out_ppn30wat.params

In [ ]:
# with h varying, tr = 1.80310435 +/- 0.01208134 , h* = 0.25997729 +/- 0.02069989
twat_n30plgapeg = []
n_bonds = 29
for i in range(len(xdata_plgapegn30wat)):
    twat_n30plgapeg.append(zimm_relax_fit(xdata_plgapegn30wat[i], 1.80568118, 0.26,  n_bonds))


In [ ]:
t_c, tz_n30plgapeg, zimm_n30plgapeg, rouse_n30plgapeg = zimm_relax_func(1, 1.80568118, 0.26,  n_bonds)

In [ ]:
x_pos_n16 = np.arange(15)
x_pos_n20 = np.arange(19)
x_pos_n30 = np.arange(29)
#x_pos_n30 = np.arange(29)
width = 0.25
aa_types_n30 = [r"$\tau_{1}$", r"$\tau_{2}$", r"$\tau_{3}$", r"$\tau_{4}$", r"$\tau_{5}$",r"$\tau_{6}$",r"$\tau_{7}$",
           r"$\tau_{8}$", r"$\tau_{9}$",r"$\tau_{10}$", r"$\tau_{11}$", r"$\tau_{12}$", r"$\tau_{13}$", r"$\tau_{14}$",
            r"$\tau_{15}$",r"$\tau_{16}$",r"$\tau_{17}$", r"$\tau_{18}$", r"$\tau_{19}$",r"$\tau_{20}$", r"$\tau_{21}$", r"$\tau_{12}$", r"$\tau_{13}$", r"$\tau_{14}$",
            r"$\tau_{25}$",r"$\tau_{26}$",r"$\tau_{27}$", r"$\tau_{28}$", r"$\tau_{29}$"]
fig = plt.figure(figsize=(10,10))
plt.title('PLGA-PEG in Water', fontsize=18)
plt.bar(x_pos_n16+(2*width), tz_n16plgapeg, width, ecolor='black',capsize=5, color='maroon', label='N = 16', align='center')
plt.bar(x_pos_n20+(width), tz_n20plgapeg, width, ecolor='black',capsize=5, color='orangered', label='N = 20', align='center')
plt.bar(x_pos_n30, tz_n30plgapeg, width, ecolor='black',capsize=5, color='darkolivegreen', label='N = 30', align='center')
plt.xticks(x_pos_n30, labels=aa_types_n30, fontsize=12)
plt.legend(fontsize=12 ,frameon=False)
plt.autoscale(enable=True, axis='both')
plt.ylabel(r'Relaxation times [ns]', fontsize=15)

In [ ]:
x_pos = np.arange(29)
aa_eigen_n30 = [r"$a_{1}$", r"$a_{2}$", r"$a_{3}$", r"$a_{4}$", r"$a_{5}$",r"$a_{6}$",r"$a_{7}$",r"$a_{8}$",
           r"$a_{9}$",r"$a_{10}$", r"$a_{11}$", r"$a_{12}$", r"$a_{13}$", r"$a_{14}$",r"$a_{15}$",r"$a_{16}$",r"$a_{17}$",
           r"$a_{18}$",r"$a_{19}$",r"$a_{20}$", r"$a_{21}$", r"$a_{22}$", r"$a_{23}$", r"$a_{24}$",r"$a_{25}$",r"$a_{26}$",
            r"$a_{27}$",r"$a_{28}$",r"$a_{29}$"]
fig = plt.figure(figsize=(10,10))
plt.title('PLGA-PEG in Water N = 30', fontsize=18)
plt.bar(x_pos_n30, rouse_n30plgapeg, width,ecolor='black',capsize=5, color='c', label='Rouse')
plt.bar(x_pos_n30+width, zimm_n30plgapeg, width,ecolor='black',capsize=5, color='m', label='Zimm')
plt.xticks(x_pos_n30+width,labels=aa_eigen_n30, fontsize=12)
plt.autoscale(enable=True, axis='both')
plt.legend(fontsize=12 ,frameon=False)
plt.ylabel(r'Rouse and Zimm eigenvalues', fontsize=15)

In [ ]:
plt.figure(figsize=(8,8))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, ydata_plgapegn6wat, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, ydata_plgapegn8wat, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, ydata_plgapegn10wat, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, ydata_plgapegn16wat, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20wat, ydata_plgapegn20wat, color='orangered', s=s_n20)
plt.scatter(xdata_plgapegn30wat, ydata_plgapegn30wat, color='darkolivegreen', s=s_n30)
plt.plot(xdata_plgapegn6wat, twat_n6plgapeg, color='teal')
plt.plot(xdata_plgapegn8wat, twat_n8plgapeg, color='dodgerblue')
plt.plot(xdata_plgapegn10wat, twat_n10plgapeg, color='darkmagenta')
plt.plot(xdata_plgapegn16wat, twat_n16plgapeg, color='maroon')
plt.plot(xdata_plgapegn20wat, twat_n20plgapeg, color='orangered')
plt.plot(xdata_plgapegn30wat, twat_n30plgapeg, color='darkolivegreen')
plt.title(r'Positional bead autocorrelation PLGA-PEG in water', fontsize=18, y=1.03)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('C(t)', fontsize=18)
#plt.legend(fontsize=14, frameon=False)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.yscale('symlog', linthreshy=0.5)
plt.xscale('symlog')
plt.grid(b=True)
plt.xlim(0,150)
plt.ylim(-1,1)
plt.tick_params(labelsize=18, direction='in', which='both', width=1, length=10)

In [ ]:
rppwat_msen30 = np.array([ydata_plgapegn30wat - np.array(twat_n30plgapeg)])
#rpwat_msen6

In [ ]:
plt.figure(figsize=(7,7))
plt.rcParams["font.family"] = "Arial"
plt.scatter(xdata_plgapegn6wat, rppwat_msen6, color='teal', s=s_n6)
plt.scatter(xdata_plgapegn8wat, rppwat_msen8, color='dodgerblue', s=s_n8)
plt.scatter(xdata_plgapegn10wat, rppwat_msen10, color='darkmagenta', s=s_n10)
plt.scatter(xdata_plgapegn16wat, rppwat_msen16, color='maroon', s=s_n16)
plt.scatter(xdata_plgapegn20wat, rppwat_msen20, color='orangered', s=s_n20)
plt.scatter(xdata_plgapegn30wat, rppwat_msen30, color='darkolivegreen', s=s_n30)
plt.title(r'Relaxation time Fitting Residuals PLGA-PEG in water', fontsize=18, y=1.01)
plt.xlabel(r'Time lag [ns]', fontsize=18)
plt.ylabel('Residuals', fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlim([0,200])
plt.ylim([-0.7,1])
plt.tick_params(labelsize=15)

#### Relaxation Times Alpha Exponent

In [ ]:
out_ppn6wat.params

In [ ]:
out_ppn8wat.params

In [ ]:
out_ppn10wat.params

In [ ]:
out_ppn16wat.params

In [ ]:
out_ppn20wat.params

In [ ]:
out_ppn30wat.params

In [ ]:
zimm_t1_plgapegwat = np.zeros(shape=(5,6))
zimm_t1_plgapegwat[3] = np.array([6,8,10,16,20,30])
zimm_t1_plgapegwat[0] = np.array([0.25745834, 0.39577315, 0.49318925, 0.88882196, 1.13179195, 1.80568118])
zimm_t1_plgapegwat[1] = np.array([0.00128257, 0.00187107, 0.00336617, 0.00497893, 0.00679416, 0.00966734])
zimm_t1_plgapegwat

In [ ]:
np.log10(zimm_t1_plgapegwat[0])

In [ ]:
np.log10(zimm_t1_plgapegwat[3])

In [ ]:
zimm_t1_plgapegwat[1]

In [ ]:
#Error Propagation: https://terpconnect.umd.edu/~toh/models/ErrorPropagation.pdf
t1plgapegwat_logerr = np.zeros(shape=(6))
for i in range(6):
    t1plgapegwat_logerr[i] = 0.434*((zimm_t1_plgapegwat[1][i])/(zimm_t1_plgapegwat[0][i]))
    
t1plgapegwat_logerr

In [ ]:
zimm_t1_plgapegwat[2] = t1plgapegwat_logerr

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(np.log10(zimm_t1_plgapegwat[3]), np.log10(zimm_t1_plgapegwat[0]), yerr=zimm_t1_plgapegwat[2], color='#0371EC', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
#plt.plot(np.log10(n_plgapeg), gg_wat, color='#A58262')

In [ ]:
# From fitting all points, I get best fit 
from sklearn.linear_model import LinearRegression
model_alplgapegwat = LinearRegression(fit_intercept=True)
model_alplgapegwat.fit(np.log10(zimm_t1_plgapegwat[3]).reshape(-1,1), np.log10(zimm_t1_plgapegwat[0]))
# Slope here is in nanometers
print("Model slope:    ", model_alplgapegwat.coef_[0])
print("Model intercept:", model_alplgapegwat.intercept_)

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
pair_regression(np.log10(zimm_t1_plgapegwat[3]),np.log10(zimm_t1_plgapegwat[0]))

In [ ]:
ggal_plgapegwat = model_alplgapegwat.predict(np.log10(zimm_t1_plgapegwat[3]).reshape(-1,1))
ggal_plgapegwat

In [ ]:
np.log10(zimm_t1_plgapegwat[0])

In [ ]:
# Residuals between the true y data and model y data 
resid_alppwat = np.log10(zimm_t1_plgapegwat[0]) - ggal_plgapegwat
resid_alppwat

In [ ]:
# How to calculate mean squared error, N - 2 is dof 
mse_alppwat = np.sum(resid_alppwat**2)/(len(resid_alppwat) - 2)
mse_alppwat

In [ ]:
# Normalized root mean squared error
np.sqrt(mse_alppwat)/(np.max(np.log10(zimm_t1_plgapegwat[0])) - np.min(np.log10(zimm_t1_plgapegwat[0])))

In [ ]:
print("R2 score:", sklearn.metrics.r2_score(np.log10(zimm_t1_plgapegwat[0]), ggal_plgapegwat))

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
getPairStats(np.log10(zimm_t1_plgapegwat[3]),np.log10(zimm_t1_plgapegwat[0]))

In [ ]:
# How to calculate Sum((Xi - avg(X))^2): X values are the bond length values 
nt_ttplgapegwat = np.log10(zimm_t1_plgapegwat[3]) - np.mean(np.log10(zimm_t1_plgapegwat[3]))
nhui_plgapegwat = nt_ttplgapegwat**2
np.sum(nhui_plgapegwat)

In [ ]:
# How to calculate Sxy
ata_tr = np.sum((np.log10(zimm_t1_plgapegwat[0]) - np.mean(np.log10(zimm_t1_plgapegwat[0])))*(nt_ttplgapegwat))
ata_tr

In [ ]:
# Test statistic, t0
slp_esttr = ata_tr/np.sum(nhui_plgapegwat)
slp_esttr/np.sqrt(mse_alppwat/(np.sum(nhui_plgapegwat)))

In [ ]:
# t-value with 95 % confidence intervals (alpha/2, n-2)
# Value of test statistic is greater than threshold, reject H0
n = len(resid_alppwat) 
alpha = 0.05
scipy.stats.t.ppf(1-alpha/2, n - 2)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_trppwat = scipy.stats.t.ppf(1-alpha/2, n - 2)*np.sqrt(mse_alppwat/np.sum(nhui_plgapegwat))
flc_trppwat

In [ ]:
# standard error of the slope
np.sqrt(mse_alppwat/np.sum(nhui_plgapegwat))

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
simple_regression_conf(np.log10(zimm_t1_plgapegwat[3]),np.log10(zimm_t1_plgapegwat[0]), alpha=0.05)

In [ ]:
zimm_t1_plgapegwat[4] = ggal_plgapegwat
zimm_t1_plgapegwat

In [ ]:
np.save('PLGAPEGwater_relax_times.npy', zimm_t1_plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(np.log10(zimm_t1_plgapegwat[3]), np.log10(zimm_t1_plgapegwat[0]), yerr=zimm_t1_plgapegwat[2], color='#0371EC', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(np.log10(zimm_t1_plgapegwat[3]), zimm_t1_plgapegwat[4], color='#0371EC')
plt.title(r'Relaxation Times Scaling Exponent, PLGA-PEG in water', fontsize=15)
plt.xlabel(r'Log($N_{PLGA-PEG}$)', fontsize=15)
plt.ylabel(r'Log($\tau_{1}$)', fontsize=15)
plt.tick_params(labelsize=14)
plt.text(0.85, 0.1, r'$\alpha_{water}$ = 1.20 ± 0.06', fontsize=15, color='#0371EC')
plt.text(0.85, 0.05, r'$R^{2}$ = 0.998, NRMSE = 0.017', fontsize=15, color='#0371EC')

#### Radius of Gyration vs. time N = 30 PLGA-PEG in water

In [ ]:
# Key variables 
n30_mon = 30
start = 0
end = 36001

In [ ]:
n30plgapeg_rgens_wat, cor_n30plgapeg_wat, n30plgapeg_cos_wat, rgwat_n30plgapeg = get_rg_pers_poly(plgapeg_n30wat, n30_plgapeg_wat,
                                                                                                  start,end)

In [ ]:
n30plgapeg_rgens_wat[0].shape

In [ ]:
rgwat_n30plgapeg

In [ ]:
n30plgapeg_rgens_wat.shape

In [ ]:
trj_len = np.arange(36001)
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG Radius of Gyration', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$R_{g}$ [nm]', fontsize=15)
#plt.plot(trj_len/100, n6plgapeg_rgens_wat[0]/10,linewidth=2, color='teal')
#plt.plot(trj_len/100, n8plgapeg_rgens_wat[0]/10,linewidth=2, color='dodgerblue')
#plt.plot(trj_len/100, n10plgapeg_rgens_wat[0]/10,linewidth=2, color='darkmagenta')
plt.plot(trj_len/100, n16plgapeg_rgens_wat[0]/10,linewidth=2, color='maroon')
plt.plot(trj_len/100, n20plgapeg_rgens_wat[0]/10,linewidth=2, color='orangered')
plt.plot(trj_len/100, n30plgapeg_rgens_wat[0]/10,linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
#plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water'],frameon=False, fontsize=14, loc='upper left')
plt.legend(['N = 16 in Water', 'N = 20 in Water','N = 30 in Water'],frameon=False, fontsize=14, loc='upper left')
#plt.legend(['N = 6 in Water', 'N = 8 in Water','N = 10 in Water','N = 16 in water','N = 20 in water','N = 30 in water'],
#           frameon=False, fontsize=14, loc='upper left')
#plt.text(127, 0.96,r'N = 6 in water', fontsize=18, color='#1F2E69', family='Arial')
plt.xlim(0,360)
plt.ylim(0.2,4)

In [ ]:
np.save('n6plgapeg_watRg.npy', n6plgapeg_rgens_wat)
np.save('n8plgapeg_watRg.npy', n8plgapeg_rgens_wat)
np.save('n10plgapeg_watRg.npy', n10plgapeg_rgens_wat)
np.save('n16plgapeg_watRg.npy', n16plgapeg_rgens_wat)
np.save('n20plgapeg_watRg.npy', n20plgapeg_rgens_wat)
np.save('n30plgapeg_watRg.npy', n30plgapeg_rgens_wat)

In [ ]:
n6plgapeg_rgens_wat = np.load('n6plgapeg_watRg.npy',allow_pickle=True)
n8plgapeg_rgens_wat = np.load('n8plgapeg_watRg.npy',allow_pickle=True)
n10plgapeg_rgens_wat = np.load('n10plgapeg_watRg.npy',allow_pickle=True)
n16plgapeg_rgens_wat = np.load('n16plgapeg_watRg.npy',allow_pickle=True)
n20plgapeg_rgens_wat = np.load('n20plgapeg_watRg.npy',allow_pickle=True)
n30plgapeg_rgens_wat = np.load('n30plgapeg_watRg.npy',allow_pickle=True)

In [ ]:
n30plgapeg_rgens_wat = np.load('n30plgapeg_watRg.npy',allow_pickle=True)
plgapegn30wat_rh = np.load('n30plgapeg_watRHYD.npy', allow_pickle=True)

In [ ]:
rg_rh_n30plgapegwat = n30plgapeg_rgens_wat[0]/plgapegn30wat_rh

In [ ]:
trj_len = np.arange(rg_rh_n30plgapegwat.shape[0])

In [ ]:
plt.figure(figsize=(7,7))
plt.plot(trj_len/100, rg_rh_n30plgapegwat)
plt.xlim(0,360)

In [ ]:
rhy_rg_plgapegwat = np.load('PLGAPEGwater_RgRh_Lc.npy', allow_pickle=True)
rhy_rg_plgapegwat[0]

In [ ]:
rhy_rg_plgapegwat[0][5]

In [ ]:
rg_rh_n30plgapegwat[999]

In [ ]:
np.where(rg_rh_n30plgapegwat < rhy_rg_plgapegwat[0][5])

#### Rg Autocorrelation

In [ ]:
ACn30plgapegwat_Rg, eqvarRg_n30plgapegwat = obs_autocorr_RA(n30plgapeg_rgens_wat[0],t_corr, window_shift,start,end)

In [ ]:
ACn30plgapegwat_Rg

In [ ]:
eqvarRg_n30plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.title(r'PLGA-PEG in water autocorrelation', fontsize=18, y=1.01)
plt.xlabel(r'Time [ns]', fontsize=15)
plt.ylabel(r'$C_{A}(t)$', fontsize=15)
plt.plot(ACn6plgapegwat_Rg[1]/100, ACn6plgapegwat_Rg[0],linewidth=2, color='teal')
plt.plot(ACn8plgapegwat_Rg[1]/100, ACn8plgapegwat_Rg[0],linewidth=2, color='dodgerblue')
plt.plot(ACn10plgapegwat_Rg[1]/100, ACn10plgapegwat_Rg[0],linewidth=2, color='darkmagenta')
plt.plot(ACn16plgapegwat_Rg[1]/100, ACn16plgapegwat_Rg[0],linewidth=2, color='maroon')
plt.plot(ACn20plgapegwat_Rg[1]/100, ACn20plgapegwat_Rg[0],linewidth=2, color='orangered')
plt.plot(ACn30plgapegwat_Rg[1]/100, ACn30plgapegwat_Rg[0],linewidth=2, color='darkolivegreen')
plt.tick_params(labelsize=14)
plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg','N = 16 Rg','N = 20 Rg','N = 30 Rg'], frameon=False, fontsize=14)
#plt.legend(['N = 6 Rg','N = 8 Rg','N = 10 Rg','N = 15 Rg'], frameon=False, fontsize=14)
#plt.legend(['N = 20 Rg','N = 30 Rg'], frameon=False, fontsize=14)
#plt.yscale('symlog', linthreshy=0.5)
#plt.xscale('symlog')
plt.xlim(0,20)
plt.ylim(-1,1)

#### Get average COM distance, arch length values

In [ ]:
cb_n30plga = np.zeros(shape=(36001))
cb_n30pplinker = np.zeros(shape=(36001))
cb_n30peg = np.zeros(shape=(36001))
count = 0
for ts in n30_plgapeg_wat.trajectory[0:36001]:
    # 1st two PLGA residues
    n30m1_plga = n30_plgapeg_wat.select_atoms("resid 1")
    n30m2_plga = n30_plgapeg_wat.select_atoms("resid 2")
    # PLGA-PEG linker (n_mon/2 to get to bPLG res no.)
    n30pl_link = n30_plgapeg_wat.select_atoms("resid 15")
    n30peg_link = n30_plgapeg_wat.select_atoms("resid 16")
    # last two PEG residues 
    n30l1_peg = n30_plgapeg_wat.select_atoms("resid 29")
    n30l2_peg = n30_plgapeg_wat.select_atoms("resid 30")
    
    cb_n30plga[count] = mda.analysis.distances.distance_array(n30m1_plga.center_of_mass(), n30m2_plga.center_of_mass(), 
                                                   box=n30_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n30pplinker[count] = mda.analysis.distances.distance_array(n30pl_link.center_of_mass(), n30peg_link.center_of_mass(), 
                                                   box=n30_plgapeg_wat.trajectory.ts.dimensions)
    
    cb_n30peg[count] = mda.analysis.distances.distance_array(n30l1_peg.center_of_mass(), n30l2_peg.center_of_mass(), 
                                                   box=n30_plgapeg_wat.trajectory.ts.dimensions)
    
    count += 1
    

    

In [ ]:
COM_n30pp = np.zeros(shape=(3,2))
COM_n30pp[0,0] = np.mean(cb_n30plga)
COM_n30pp[1,0] = np.mean(cb_n30pplinker)
COM_n30pp[2,0] = np.mean(cb_n30peg)
COM_n30pp[0,1] = np.std(cb_n30plga)
COM_n30pp[1,1] = np.std(cb_n30pplinker)
COM_n30pp[2,1] = np.std(cb_n30peg)
COM_n30pp

#### Correlation values at each arc length for the whole 360 ns trajectory, N = 30 PLGA-PEG/water

In [ ]:
# x values
blen_n30pp = np.zeros(shape=29)
for i in range(n30_mon-1):
    if i < int((n30_mon/2)-1):
        # PLGA contour length values
        if i == 0:
            blen_n30pp[i] = 0
        elif i != 0:
            blen_n30pp[i] = np.sum(blen_n30pp[i-1]) + COM_n30pp[0,0]    
    elif i == int((n30_mon/2)-1):
        #PLGA-PEG linker length
        blen_n30pp[i] = np.sum(blen_n30pp[i-1]) + COM_n30pp[1,0]
    elif i > int((n30_mon/2)-1):
    #    # PEG contour length values
        blen_n30pp[i] = np.sum(blen_n30pp[i-1]) + COM_n30pp[2,0]
        
blen_n30pp

In [ ]:
# Error prop. into natural log std deviation
mk_n30ppwat = cor_n30plgapeg_wat[1]/cor_n30plgapeg_wat[0]
mk_n30ppwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n30pp, np.log(cor_n30plgapeg_wat[0]), yerr=mk_n30ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')

In [ ]:
# All the points give the best fits for N = 6 peg in water
n30_blksplgapeg_wat , n30plgapeg_lpwat = bavg_pers_cnt(5, plgapeg_n30wat, n30_plgapeg_wat, blen_n30pp, 5, 0 , 36001)

In [ ]:
n30_blksplgapeg_wat

In [ ]:
n30plgapeg_lpwat

In [ ]:
n30plgapeg_lpwat[2]

In [ ]:
np.mean(n30plgapeg_lpwat[3])

#### line fitted to cosine theta values using mean slope

In [ ]:
blen_n30pp

In [ ]:
gg_n30plgapeg_wat = line_fit(np.mean(n30plgapeg_lpwat[2]),blen_n30pp)

In [ ]:
gg_n20plgapeg_n30wat = line_fit(np.mean(n20plgapeg_lpwat[2]),blen_n30pp)

In [ ]:
gg_n16plgapeg_n30wat = line_fit(np.mean(n16plgapeg_lpwat[2]),blen_n30pp)

In [ ]:
gg_n10plgapeg_n30wat = line_fit(np.mean(n10plgapeg_lpwat[2]),blen_n30pp)

In [ ]:
gg_n8plgapeg_n30wat = line_fit(np.mean(n8plgapeg_lpwat[2]),blen_n30pp)

In [ ]:
gg_n6plgapeg_n30wat = line_fit(np.mean(n6plgapeg_lpwat[2]),blen_n30pp)

In [ ]:
gg_n30plgapeg_wat

#### Block averaged Rg, Ree, Rh, mean sq. Ree and mean sq. Rg persistence length, N = 30 PLGA-PEG in water

In [ ]:
blppn30 = []
for i in range(n30_mon-1):
    if i < int((n30_mon/2)-1):
        # PLGA contour length values
        blppn30.append(COM_n30pp[0,0])    
    elif i == int((n30_mon/2)-1):
        #PLGA-PEG linker length
        blppn30.append(COM_n30pp[1,0]) 
    elif i > int((n30_mon/2)-1):
    #    # PEG contour length values
        blppn30.append(COM_n30pp[2,0]) 

# Contour Length, N = 6 PLGA-PEG in water
lc_ppn30 = np.sum(blppn30)
lc_ppn30

#### Mean sq. e2e/Mean sq. Rg vs contour length PLGA-PEG in water

In [ ]:
# Ree^2/Rg^2 array at different contour lengths
re2e_rgsq_plgapegwat[1,5] = lc_ppn30
re2e_rgsq_plgapegwat

In [ ]:
a_ppn30 = np.mean(n30_blksplgapeg_wat['Avg Sq. end to end distance'])

In [ ]:
b_ppn30 = np.mean(n30_blksplgapeg_wat['Avg Sq. radius of gyration'])

In [ ]:
re2e_rgsq_plgapegwat[0,5] = a_ppn30/b_ppn30
re2e_rgsq_plgapegwat

#### Error propagation: Mean sq. e2e/Mean sq. Rg

In [ ]:
e2esq_plgapegwat = np.array([a_ppn6,a_ppn8, a_ppn10, a_ppn16,a_ppn20, a_ppn30])
e2esq_plgapegwat

In [ ]:
rgsq_plgapegwat = np.array([b_ppn6,b_ppn8, b_ppn10, b_ppn16, b_ppn20, b_ppn30])
rgsq_plgapegwat

In [ ]:
# Error Propagation: https://terpconnect.umd.edu/~toh/models/ErrorPropagation.pdf
stde2e_pdsys = np.array([np.std(n6_blksplgapeg_wat['Avg Sq. end to end distance']),
                         np.std(n8_blksplgapeg_wat['Avg Sq. end to end distance']),
                        np.std(n10_blksplgapeg_wat['Avg Sq. end to end distance']),
                         np.std(n16_blksplgapeg_wat['Avg Sq. end to end distance']),
                        np.std(n20_blksplgapeg_wat['Avg Sq. end to end distance']),
                        np.std(n30_blksplgapeg_wat['Avg Sq. end to end distance'])])
stde2e_pdsys

In [ ]:
stdrgsq_pdsys = np.array([np.std(n6_blksplgapeg_wat['Avg Sq. radius of gyration']),
                         np.std(n8_blksplgapeg_wat['Avg Sq. radius of gyration']),
                         np.std(n10_blksplgapeg_wat['Avg Sq. radius of gyration']),
                          np.std(n16_blksplgapeg_wat['Avg Sq. radius of gyration']),
                         np.std(n20_blksplgapeg_wat['Avg Sq. radius of gyration']),
                         np.std(n30_blksplgapeg_wat['Avg Sq. radius of gyration'])])
stdrgsq_pdsys

In [ ]:
re2e_rgsq_plgapegwat[0]

In [ ]:
x_pdsys = np.zeros(shape=6)
for i in range(6):
    ad_s = np.sqrt((((stde2e_pdsys[i]/e2esq_plgapegwat[i])**2) + ((stdrgsq_pdsys[i]/rgsq_plgapegwat[i])**2)))
    x_pdsys[i] = re2e_rgsq_plgapegwat[0][i]*ad_s
    
x_pdsys

In [ ]:
re2e_rgsq_plgapegwat

In [ ]:
re2e_rgsq_plgapegwat[2] = x_pdsys
re2e_rgsq_plgapegwat

In [ ]:
re2e_rgsq_plgapegwat.shape

In [ ]:
np.save('PLGA-PEGwat_msRee_msRg.npy', re2e_rgsq_plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(re2e_rgsq_plgapegwat[1]/10, re2e_rgsq_plgapegwat[0], yerr=re2e_rgsq_plgapegwat[2], color='#0371EC', linestyle="-",marker='v',
     capsize=5, capthick=1, ecolor='#0371EC')
plt.title('Mean sq. Ree/Mean sq. Rg, PLGA-PEG in water', fontsize=15, y=1.01)
plt.xlabel(r'Contour Length, $L_{c}$ [nm]', fontsize=15)
plt.ylabel(r'$\left<R_{ee}^{2}\right>/\left<R_{g}^{2}\right>$', fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.xlim(1,15)
plt.ylim(2,10)

#### Rg/Rh vs contour length PLGA-PEG in water

In [ ]:
# Rg/Rh array at different contour lengths 
rhy_rg_plgapegwat[1,5] = lc_ppn30
rhy_rg_plgapegwat

In [ ]:
aa_ppn30 = np.mean(n30_blksplgapeg_wat['Avg Radius of gyration'])

In [ ]:
bb_ppn30 = np.mean(n30_blksplgapeg_wat['Avg Hydrodynamic radius'])

In [ ]:
rhy_rg_plgapegwat[0,5] = aa_ppn30/bb_ppn30
rhy_rg_plgapegwat

In [ ]:
rhy_rg_plgapegwat

#### Error Propagation: Rg/Rh vs contour length

In [ ]:
rgsys_plgapegwat = np.array([aa_ppn6,aa_ppn8, aa_ppn10, aa_ppn16,aa_ppn20, aa_ppn30])
rgsys_plgapegwat

In [ ]:
rhydsys_plgapegwat = np.array([bb_ppn6,bb_ppn8, bb_ppn10, bb_ppn16, bb_ppn20, bb_ppn30])
rhydsys_plgapegwat

In [ ]:
# Error Propagation: https://terpconnect.umd.edu/~toh/models/ErrorPropagation.pdf
stdrg_pdsys = np.array([np.std(n6_blksplgapeg_wat['Avg Radius of gyration']),
                         np.std(n8_blksplgapeg_wat['Avg Radius of gyration']),
                        np.std(n10_blksplgapeg_wat['Avg Radius of gyration']),
                        np.std(n16_blksplgapeg_wat['Avg Radius of gyration']),
                        np.std(n20_blksplgapeg_wat['Avg Radius of gyration']),
                        np.std(n30_blksplgapeg_wat['Avg Radius of gyration'])])
stdrg_pdsys

In [ ]:
std_rhydsq_pdsys = np.array([np.std(n6_blksplgapeg_wat['Avg Hydrodynamic radius']),
                         np.std(n8_blksplgapeg_wat['Avg Hydrodynamic radius']),
                         np.std(n10_blksplgapeg_wat['Avg Hydrodynamic radius']),
                          np.std(n16_blksplgapeg_wat['Avg Hydrodynamic radius']),    
                         np.std(n20_blksplgapeg_wat['Avg Hydrodynamic radius']),
                         np.std(n30_blksplgapeg_wat['Avg Hydrodynamic radius'])])
std_rhydsq_pdsys

In [ ]:
xrgrh_pdsys = np.zeros(shape=6)
for i in range(6):
    adn1_s = np.sqrt((((stdrg_pdsys[i]/rgsys_plgapegwat[i])**2) + ((std_rhydsq_pdsys[i]/rhydsys_plgapegwat[i])**2)))
    xrgrh_pdsys[i] = rhy_rg_plgapegwat[0][i]*adn1_s
    
xrgrh_pdsys

In [ ]:
rhy_rg_plgapegwat

In [ ]:
rhy_rg_plgapegwat[2] = xrgrh_pdsys
rhy_rg_plgapegwat

In [ ]:
np.save('PLGAPEGwater_RgRh_Lc.npy', rhy_rg_plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(rhy_rg_plgapegwat[1]/10, rhy_rg_plgapegwat[0], yerr=rhy_rg_plgapegwat[2], color='#0371EC', linestyle="-",
             marker='^', capsize=5, capthick=1, ecolor='#0371EC')
plt.title(r'Rg/Rh PLGA-PEG in water', fontsize=15, y=1.01)
plt.xlabel(r'Contour Length, $L_{c}$ [nm]', fontsize=15)
plt.ylabel(r'$\left<R_{g}\right>/\left<R_{hyd}\right>$', fontsize=15)
plt.xlim(1,15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.ylim(0.3,0.7)

### Kuhn lenght vs monomer length

In [ ]:
REElclk_plgapegwat = np.load('PLGAPEGwater_REElclk.npy', allow_pickle=True)

In [ ]:
REElclk_plgapegwat = np.zeros(shape=(4,6))
REElclk_plgapegwat[0] = e2esq_plgapegwat
REElclk_plgapegwat[1] = rhy_rg_plgapegwat[1]
REElclk_plgapegwat[2] = stde2e_pwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(REElclk_plgapegwat[1], REElclk_plgapegwat[0], color='#0371EC', yerr=REElclk_plgapegwat[2], linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='#0371EC')

In [ ]:
# Linear fitting procedure: https://www2.isye.gatech.edu/~yxie77/isye2028/lecture12.pdf
ne2e = REElclk_plgapegwat[1]
from sklearn.linear_model import LinearRegression
model_e2eppwat = LinearRegression(fit_intercept=True)
model_e2eppwat.fit(ne2e.reshape(-1,1), REElclk_plgapegwat[0])
# Slope here is in angstroms
print("Model slope:    ", model_e2eppwat.coef_[0])
print("Model intercept:", model_e2eppwat.intercept_)

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
pair_regression(REElclk_plgapegwat[1],REElclk_plgapegwat[0])

In [ ]:
gge2e_ppwat = model_e2eppwat.predict(ne2e.reshape(-1,1))
gge2e_ppwat

In [ ]:
REElclk_plgapegwat[0]

In [ ]:
# Residuals between the true y data and model y data 
resid_e2eppwat = REElclk_plgapegwat[0] - gge2e_ppwat
resid_e2eppwat

In [ ]:
# How to calculate mean squared error, N - 2 is dof 
mse_e2eppwat = np.sum(resid_e2eppwat**2)/(len(resid_e2eppwat) - 2)
mse_e2eppwat

In [ ]:
# Normalized root mean squared error
np.sqrt(mse_e2eppwat)/(np.max(REElclk_plgapegwat[0]) - np.min(REElclk_plgapegwat[0]))

In [ ]:
print("R2 score:", sklearn.metrics.r2_score(REElclk_plgapegwat[0].reshape(-1,1), gge2e_ppwat))

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
getPairStats(REElclk_plgapegwat[1],REElclk_plgapegwat[0])

In [ ]:
# How to calculate Sxx
nt_tte2epd = ne2e - np.mean(ne2e)
nhui_e2epd = nt_tte2epd**2
np.sum(nhui_e2epd)

In [ ]:
# How to calculate Sxy
ata = np.sum((REElclk_plgapegwat[0] - np.mean(REElclk_plgapegwat[0]))*(ne2e - np.mean(ne2e)))
ata

In [ ]:
# Test statistic, t0
(ata/np.sum(nhui_e2epd))/np.sqrt(mse_e2eppwat/(np.sum(nhui_e2epd)))

In [ ]:
# t-value with 95 % confidence intervals (alpha, n-2)
# Value of test statistic is greater than threshold, reject H0
n = len(resid_e2eppwat) 
alpha = 0.05
scipy.stats.t.ppf(1-alpha/2, n - 2)
#scipy.stats.t.ppf(0.95, len(resid_e2epgwat) - 1)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_e2epd = scipy.stats.t.ppf(1-alpha/2, n - 2)*np.sqrt(mse_e2eppwat/(np.sum(nhui_e2epd)))
flc_e2epd

In [ ]:
# standard error of the slope
np.sqrt(mse_e2eppwat/np.sum(nhui_e2epd))

In [ ]:
# https://towardsdatascience.com/how-to-perform-linear-regression-with-confidence-5ed8fc0bb9fe
simple_regression_conf(REElclk_plgapegwat[1],REElclk_plgapegwat[0], alpha=0.05)

In [ ]:
REElclk_plgapegwat[3] = gge2e_ppwat
REElclk_plgapegwat

In [ ]:
np.save('PLGAPEGwater_REElclk.npy', REElclk_plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(REElclk_plgapegwat[1], REElclk_plgapegwat[0], color='#0371EC', yerr=REElclk_plgapegwat[2], linestyle="None",marker='v',
     capsize=5, capthick=1, ecolor='#0371EC')
plt.plot(REElclk_plgapegwat[1], REElclk_plgapegwat[3], color='#0371EC')
plt.title(r'Kuhn Length, PLGA-PEG in water', fontsize=15)
plt.ylabel(r'$\left<R_{ee}^{2}\right>$', fontsize=15)
plt.xlabel(r'Contour Length, $L_{c}$ [$\AA$]', fontsize=15)
plt.tick_params(labelsize=14)
plt.text(30, 600, r'$l^{water}_{k}$ = 3.13 $\AA$ ± 0.20 $\AA$', fontsize=15, color='#0371EC')
plt.text(30, 560, r'$R^{2}$ = 0.983, NRMSE = 0.054', fontsize=15, color='#0371EC')
plt.xlim(10,150)
plt.ylim(0,700)

#### Lp alternate method

In [ ]:
REElclk_plgapegwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(REElclk_plgapegwat[1], REElclk_plgapegwat[0], color='#0371EC', yerr=REElclk_plgapegwat[2], linestyle="None",marker='v',
     capsize=5, capthick=1, ecolor='#0371EC')
#plt.plot(REElclk_plgapegwat[1]/10, REElclk_plgapegwat[3], color='#0371EC')
plt.title(r'Kuhn Length, PLGA-PEG in water', fontsize=15)
plt.ylabel(r'$\left<R_{ee}^{2}\right>$', fontsize=15)
plt.xlabel(r'Contour Length, $L_{c}$ [A]', fontsize=15)
plt.tick_params(labelsize=14)
plt.xlim(10,150)
plt.ylim(0,700)

In [ ]:
REElclk_plgapegwat[2]

In [ ]:
REElclk_plgapegwat[1]

In [ ]:
REElclk_plgapegwat[0]

In [ ]:
def sqree_lpfitting(lc, lp):
    
    fir_p = 2*lp*lc
    
    sec_p = 1 - ((lp/lc)*(1 - np.exp(-lc/lp)))
    
    return fir_p*sec_p
    

In [ ]:
sqree_lpfitting(16.2, 3.8)

In [ ]:
def perslen_fitplgapegwat(varbs, xlc, yree):
    fir_varb = varbs['lp']
    test_yree = []
    
    for i in range(len(xlc)):
        model_yree = sqree_lpfitting(xlc[i],fir_varb)
        test_yree.append(yree[i]- model_yree)
        
    return np.array(test_yree)

In [ ]:
from lmfit import Minimizer, Parameters, report_fit

In [ ]:
plgapegwat_perfit = Parameters()
plgapegwat_perfit.add(name='lp', value=0)

In [ ]:
perlc_plgapegwat = Minimizer(perslen_fitplgapegwat, plgapegwat_perfit, fcn_args=(REElclk_plgapegwat[1], REElclk_plgapegwat[0]))
olplc_plgapegwat = perlc_plgapegwat.leastsq()
#bfit_n10 = ydata_n10wat + out_n10wat.residual 

In [ ]:
report_fit(olplc_plgapegwat.params)

In [ ]:
olplc_plgapegwat.params

In [ ]:
plgapegwatlcree = []
xlc = REElclk_plgapegwat[1]
for i in range(len(REElclk_plgapegwat[1])):
    plgapegwatlcree.append(sqree_lpfitting(xlc[i], 2.04992305))
    
md_nlppwat = np.array(plgapegwatlcree)

In [ ]:
md_nlppwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(REElclk_plgapegwat[1], REElclk_plgapegwat[0], color='#0371EC', yerr=REElclk_plgapegwat[2], linestyle="None",marker='v',
    capsize=5, capthick=1, ecolor='#0371EC')
plt.plot(REElclk_plgapegwat[1], md_nlppwat, color='r', ls='--')
plt.title(r'$L_{p}$ alt calc, PLGA-PEG in water', fontsize=15)
plt.ylabel(r'$\left<R_{ee}^{2}\right>$', fontsize=15)
plt.xlabel(r'Contour Length, $L_{c}$ [$\AA$]', fontsize=15)
plt.legend([r'Model: $L_{p}$ = 2.05 $\AA$ ± 0.15 $\AA$','Sim'], frameon=False, fontsize=14)
plt.tick_params(labelsize=14)
plt.xlim(10,150)
plt.ylim(0,700)

#### Error Propagation: Lp/Lc ratio

In [ ]:
# pers length/contour length 
kuhnlpL_plgapegwat[0,5] = np.mean(n30_blksplgapeg_wat["Avg persistence length"])/lc_ppn30
kuhnlpL_plgapegwat

In [ ]:
kuhnlpL_plgapegwat

In [ ]:
lpsys_plgapegwat = np.array([np.mean(n6_blksplgapeg_wat["Avg persistence length"]),
                         np.mean(n8_blksplgapeg_wat["Avg persistence length"]),
                        np.mean(n10_blksplgapeg_wat["Avg persistence length"]),
                        np.mean(n16_blksplgapeg_wat["Avg persistence length"]),  
                        np.mean(n20_blksplgapeg_wat["Avg persistence length"]),
                        np.mean(n30_blksplgapeg_wat["Avg persistence length"])])
lpsys_plgapegwat

In [ ]:
# Error Propagation: https://terpconnect.umd.edu/~toh/models/ErrorPropagation.pdf
stdlp_pdsys = np.array([np.std(n6_blksplgapeg_wat["Avg persistence length"]),
                         np.std(n8_blksplgapeg_wat["Avg persistence length"]),
                        np.std(n10_blksplgapeg_wat["Avg persistence length"]),
                        np.std(n16_blksplgapeg_wat["Avg persistence length"]),
                        np.std(n20_blksplgapeg_wat["Avg persistence length"]),
                        np.std(n30_blksplgapeg_wat["Avg persistence length"])])
stdlp_pdsys

In [ ]:
xlp_pdsys = np.zeros(shape=6)
for i in range(6):
    adn2_s = np.sqrt((((stdlp_pdsys[i]/lpsys_plgapegwat[i])**2)))
    xlp_pdsys[i] = kuhnlpL_plgapegwat[0][i]*adn2_s
    
xlp_pdsys

In [ ]:
kuhnlpL_plgapegwat[2] = xlp_pdsys
kuhnlpL_plgapegwat

In [ ]:
np.save('PLGAPEGwater_lk_LpL.npy', kuhnlpL_plgapegwat)

In [ ]:
plt.figure(figsize=(7,7))
plt.xlabel(r'$N_{PEG}$', fontsize=15)
plt.ylabel(r'$\left<L_{p}\right>/L_{c}$', fontsize=15)
plt.title('Level of rigidity, PLGA-PEG in water', fontsize=15)
plt.errorbar(kuhnlpL_plgapegwat[1], kuhnlpL_plgapegwat[0], yerr=kuhnlpL_plgapegwat[2], marker='^',capsize=5, capthick=1)
plt.tick_params(axis='both',labelsize=15, width=1)
plt.ylim(0,1.5)
plt.xlim(5,35)

### Block averaged Radius of gyration and persistence length, N = 30 PLGA-PEG in water

In [ ]:
np.mean(n30_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.std(n30_blksplgapeg_wat["Avg persistence length"])

In [ ]:
np.mean(n30_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
np.std(n30_blksplgapeg_wat["Avg Radius of gyration"])

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(blen_n6pp, np.log(cor_n6plgapeg_wat[0]), yerr=mk_n6ppwat, color='teal', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n8pp, np.log(cor_n8plgapeg_wat[0]), yerr=mk_n8ppwat, color='dodgerblue', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n10pp, np.log(cor_n10plgapeg_wat[0]), yerr=mk_n10ppwat, color='darkmagenta', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n16pp, np.log(cor_n16plgapeg_wat[0]), yerr=mk_n16ppwat, color='maroon', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n20pp, np.log(cor_n20plgapeg_wat[0]), yerr=mk_n20ppwat, color='orangered', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.errorbar(blen_n30pp, np.log(cor_n30plgapeg_wat[0]), yerr=mk_n30ppwat, color='darkolivegreen', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(blen_n30pp, gg_n6plgapeg_n30wat, color='teal')
plt.plot(blen_n30pp, gg_n8plgapeg_n30wat, color='dodgerblue')
plt.plot(blen_n30pp, gg_n10plgapeg_n30wat, color='darkmagenta')
plt.plot(blen_n30pp, gg_n16plgapeg_n30wat, color='orangered')
plt.plot(blen_n30pp, gg_n20plgapeg_n30wat, color='maroon')
plt.plot(blen_n30pp, gg_n30plgapeg_wat, color='darkolivegreen')
plt.title(r'Ensemble Averaged ln(Cosine $\theta$) PLGA-PEG in water', fontsize=15, y=1.01)
plt.xlabel(r'Bond Length', fontsize=15)
plt.ylabel(r'ln$\left< Cos(\theta)\right >$', fontsize=15)
plt.ylim(-7,1.5)
plt.xlim(0,125)
#plt.ylim(-1.9,0)
font = font_manager.FontProperties(family='Arial', style='normal', size='14')
#plt.legend([r'$N_{PEG}$ = 6: $L_{p}$ = 18.8 $\AA$ ± 3.60 $\AA$'], loc=3, frameon=0, fontsize=14, prop=font)
plt.tick_params(labelsize=14)
plt.text(0.5, -6.9,r'$N_{PLGA-PEG}$ = 6: $L_{p}$ = 19.4 $\AA$ ± 1.43 $\AA$', fontsize=15, color='teal')
plt.text(0.5, -6.58,r'$N_{PLGA-PEG}$ = 8: $L_{p}$ = 20.8 $\AA$ ± 1.03 $\AA$', fontsize=15, color='dodgerblue')
plt.text(0.5, -6.23,r'$N_{PLGA-PEG}$ = 10: $L_{p}$ = 21.2 $\AA$ ± 0.58 $\AA$', fontsize=15, color='darkmagenta')
plt.text(0.5, -5.90,r'$N_{PLGA-PEG}$ = 16: $L_{p}$ = 23.1 $\AA$ ± 0.86 $\AA$', fontsize=15, color='orangered')
plt.text(0.5, -5.50,r'$N_{PLGA-PEG}$ = 20: $L_{p}$ = 23.2 $\AA$ ± 0.60 $\AA$', fontsize=15, color='maroon')
plt.text(0.5, -5.10,r'$N_{PLGA-PEG}$ = 30: $L_{p}$ = 25.2 $\AA$ ±4.68 $\AA$', fontsize=15, color='darkolivegreen')

In [ ]:
rgplgapeg_olig_wat[r"$R_{g}$ [Angstrom] N = 30 PEG wat "] = n30_blksplgapeg_wat["Avg Radius of gyration"]
rgplgapeg_olig_wat

In [ ]:
pers_plgapegt_wat[r"$L_{p}$ [Angstrom] N = 30 PEG wat "] = n30_blksplgapeg_wat["Avg persistence length"]
pers_plgapegt_wat

In [ ]:
rgplgapeg_olig_wat.to_pickle("PLGAPEG_wat_Rg.pkl")

In [ ]:
pers_plgapegt_wat.to_pickle("PLGAPEG_wat_Lp.pkl")

### Fluory Exponent, PLGA-PEG in water systems 

In [ ]:
n_plgapeg = np.array([6,8,10,16,20,30])
rg_nplgapeg_wat = np.array([np.mean(n6_blksplgapeg_wat["Avg Radius of gyration"])
                    ,np.mean(n8_blksplgapeg_wat["Avg Radius of gyration"]),np.mean(n10_blksplgapeg_wat["Avg Radius of gyration"])
                    ,np.mean(n16_blksplgapeg_wat["Avg Radius of gyration"]),
                    np.mean(n20_blksplgapeg_wat["Avg Radius of gyration"]),np.mean(n30_blksplgapeg_wat["Avg Radius of gyration"])])

In [ ]:
rg_nplgapeg_wat

In [ ]:
rgwat_nplgapeg_std = np.array([np.std(np.log10(n6_blksplgapeg_wat["Avg Radius of gyration"]))
                    ,np.std(np.log10(n8_blksplgapeg_wat["Avg Radius of gyration"]))
                        ,np.std(np.log10(n10_blksplgapeg_wat["Avg Radius of gyration"]))
                       ,np.std(np.log10(n16_blksplgapeg_wat["Avg Radius of gyration"]))     
                    ,np.std(np.log10(n20_blksplgapeg_wat["Avg Radius of gyration"]))
                        ,np.std(np.log10(n30_blksplgapeg_wat["Avg Radius of gyration"]))])
rgwat_nplgapeg_std

In [ ]:
n_plgapeg

In [ ]:
np.log10(rg_nplgapeg_wat)

In [ ]:
np.log10(n_plgapeg)

In [ ]:
# From fitting all points, I get best fit 
from sklearn.linear_model import LinearRegression
model_vwat = LinearRegression(fit_intercept=True)
model_vwat.fit(np.log10(n_plgapeg).reshape(-1,1), np.log10(rg_nplgapeg_wat))
# Slope here is in nanometers
print("Model slope:    ", model_vwat.coef_[0])
print("Model intercept:", model_vwat.intercept_)

In [ ]:
gg_wat = model_vwat.predict(np.log10(n_plgapeg).reshape(-1,1))
gg_wat

In [ ]:
print("Mean Std Error:", sklearn.metrics.mean_squared_error(np.log10(rg_nplgapeg_wat), gg_wat))
print("R2 score:", sklearn.metrics.r2_score(np.log10(rg_nplgapeg_wat), gg_wat))

In [ ]:
# Residuals between the true y data and model y data 
resid_vwat = np.log10(rg_nplgapeg_wat) - gg_wat
resid_vwat

In [ ]:
# How to calculate Sum((Xi - avg(X))^2): X values are the bond length values 
nt_ttwat = np.log10(n_plgapeg)
nt_ttwat -= np.mean(nt_ttwat)
nhui_wat = nt_ttwat**2
np.sum(nhui_wat)

In [ ]:
# t-value with 95 % confidence intervals 
scipy.stats.t.ppf(0.95, 4)

In [ ]:
# How to calculate 95% confidence interval for the slope 
flc_vwat = scipy.stats.t.ppf(0.95, 4)*np.sqrt((np.sum(resid_vwat**2)/(len(resid_vwat)-1))/(np.sum(nhui_wat)))
flc_vwat

In [ ]:
plt.figure(figsize=(7,7))
plt.errorbar(np.log10(n_plgapeg), np.log10(rg_nplgapeg_wat), yerr=rgwat_nplgapeg_std, color='#A58262', linestyle="None",marker='o',
     capsize=5, capthick=1, ecolor='black')
plt.plot(np.log10(n_plgapeg), gg_wat, color='#A58262')
plt.title(r'Fluory Exponent', fontsize=15)
plt.xlabel(r'Log($N_{PLGA-PEG}$)', fontsize=15)
plt.ylabel(r'Log($R_{g}$)', fontsize=15)
plt.tick_params(labelsize=14)
plt.text(1.1, 0.75, r'$v_{water}$ = 0.40 ± 0.03', fontsize=15, color='#A58262')